In [1]:
from glob import glob
from os import listdir, makedirs
from os.path import join
import pandas as pd
import numpy as np

# Data set 1: Yatsunenko et al

In [2]:
group_sig_out_dir = './output/distance'

## Import data

Get a list of all the files in the output group significance directory

In [3]:
taxa = listdir(group_sig_out_dir)

Iterate over each of the output files, and import the group significance results

In [4]:
# grab tsv files
bact_dfs = []
for d in taxa:
    data_fp = join(group_sig_out_dir,
                   d,
                   'permanova',
                   'raw_data.tsv')
    
    d_df = pd.read_csv(data_fp, 
                       header=0,
                       sep='\t',
                       index_col=0)

    d_df['Bacterial_Genus'] = d
    
    bact_dfs.append(d_df)
    

# load group significance tsvs


Combine these results into a single dataframe.

In [5]:
combined_df = pd.concat(bact_dfs, ignore_index=True)

In [6]:
combined_df.head()

,SubjectID1,SubjectID2,Group1,Group2,Distance,Bacterial_Genus
0,100030.13393.SRR8978344,100030.13393.SRR8978339,chimp_DRC_wild_campbell,chimp_DRC_wild_campbell,1.0,k__Bacteria_p___c___o___f___g__
1,100030.13393.SRR8978339,65716.12173.133,chimp_DRC_wild_campbell,chimp_TZA_wild_moeller,1.0,k__Bacteria_p___c___o___f___g__
2,100030.13393.SRR8978339,65716.12173.104,chimp_DRC_wild_campbell,chimp_TZA_wild_moeller,0.5,k__Bacteria_p___c___o___f___g__
3,100030.13393.SRR8978339,65716.12173.144,chimp_DRC_wild_campbell,chimp_TZA_wild_moeller,1.0,k__Bacteria_p___c___o___f___g__
4,100030.13393.SRR8978339,65716.12173.150,chimp_DRC_wild_campbell,chimp_TZA_wild_moeller,1.0,k__Bacteria_p___c___o___f___g__


## Aggregate distance values

Using the Pandas pivot table functionality, aggregate the individual dissimilarity comparisons by group using `np.mean`.

In [7]:
combined_pivot = combined_df.pivot_table(values='Distance',
                        index=['Group1', 'Group2'],
                        columns=['Bacterial_Genus'],
                        aggfunc=np.mean)

In [8]:
combined_pivot.reset_index(inplace=True)

In [9]:
combined_pivot.head()


Bacterial_Genus,Group1,Group2,Unassigned_____,k__Archaea_p__Crenarchaeota_c__Thaumarchaeota_o__Nitrososphaerales_f__Nitrososphaeraceae_g__CandidatusNitrososphaera,k__Archaea_p__Euryarchaeota_c__Methanobacteria_o__Methanobacteriales_f__Methanobacteriaceae_g__Methanobrevibacter,k__Archaea_p__Euryarchaeota_c__Methanobacteria_o__Methanobacteriales_f__Methanobacteriaceae_g__Methanosphaera,k__Archaea_p__Euryarchaeota_c__Thermoplasmata_o__E2_f__[Methanomassiliicoccaceae]_g__vadinCA11,k__Bacteria_____,k__Bacteria_p__Acidobacteria_c__Acidobacteria-6_o__iii1-15_f___g__,k__Bacteria_p__Acidobacteria_c__Acidobacteriia_o__Acidobacteriales_f__Acidobacteriaceae_g__,...,k__Bacteria_p__Tenericutes_c__Mollicutes_o__Anaeroplasmatales_f__Anaeroplasmataceae_g__,k__Bacteria_p__Tenericutes_c__Mollicutes_o__Anaeroplasmatales_f__Anaeroplasmataceae_g__Anaeroplasma,k__Bacteria_p__Tenericutes_c__Mollicutes_o__Mycoplasmatales_f__Mycoplasmataceae_g__,k__Bacteria_p__Tenericutes_c__Mollicutes_o__RF39_f___g__,k__Bacteria_p__Tenericutes_c__RF3_o__ML615J-28_f___g__,k__Bacteria_p__Verrucomicrobia_c__Opitutae_o__HA64_f___g__,k__Bacteria_p__Verrucomicrobia_c__Opitutae_o__[Cerasicoccales]_f__[Cerasicoccaceae]_g__,k__Bacteria_p__Verrucomicrobia_c__Verruco-5_o__WCHB1-41_f__RFP12_g__,k__Bacteria_p__Verrucomicrobia_c__Verrucomicrobiae_o__Verrucomicrobiales_f__Verrucomicrobiaceae_g__Akkermansia,k__Bacteria_p___c___o___f___g__
0,chimp_DRC_wild_campbell,chimp_DRC_wild_campbell,0.920635,NaN,NaN,NaN,0.200000,0.846254,0.333333,1.000000,...,NaN,NaN,NaN,0.632296,1.000000,NaN,0.135761,0.646717,0.553968,1.000000
1,chimp_DRC_wild_campbell,chimp_TZA_wild_moeller,1.000000,NaN,NaN,0.197619,0.349583,0.962346,0.852381,NaN,...,NaN,NaN,NaN,0.795701,0.784660,NaN,0.430928,0.705881,0.405233,0.821429
2,chimp_DRC_wild_campbell,chimp_USA_captive_campbell,0.928571,NaN,NaN,0.285185,0.562698,0.969989,NaN,NaN,...,NaN,NaN,NaN,0.873083,0.855688,NaN,1.000000,1.000000,0.847937,NaN
3,chimp_DRC_wild_campbell,chimp_USA_captive_moeller,NaN,NaN,NaN,0.217949,0.561645,0.983341,NaN,0.510417,...,NaN,NaN,NaN,0.895034,0.896667,NaN,1.000000,1.000000,0.839608,NaN
4,chimp_DRC_wild_campbell,douc_SGP_captive_clayton,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,0.500000,...,NaN,NaN,NaN,0.991665,1.000000,NaN,NaN,1.000000,NaN,NaN


## Load target group comparisons

This file should list just the specific group comparisons we want to look at.

In [10]:
target_grp_df = pd.read_csv('./data/host_specificity_score_populations.txt', sep='\t')

In [11]:
target_grp_df

,captive_population,human_population,wild_population
0,douc_USA_captive_clayton,human_USA,douc_VNM_wild_clayton
1,douc_SGP_captive_clayton,human_USA,douc_VNM_wild_clayton
2,chimp_USA_captive_moeller,human_USA,chimp_TZA_wild_moeller
3,chimp_USA_captive_campbell,human_USA,chimp_DRC_wild_campbell
4,howler_CRI_captive_clayton,human_USA,howler_CRI_wild_clayton
5,gorilla_USA_captive_campbell,human_USA,gorilla_DRC_wild_campbell
6,douc_VNM_semicaptive_clayton,human_USA,douc_VNM_wild_clayton


## Filter data list to target comparisons

Update the aggregated pivot table to include the comparison number and comparison direction (captive to human or captive to wild) as additional columns

In [12]:
combined_pivot['comparison_num'] = 'None'
combined_pivot['comparison_dir'] = 'None'

for i, row in target_grp_df.iterrows():
    combined_pivot.loc[(combined_pivot['Group1'] == row['captive_population']) &
                       (combined_pivot['Group2'] == row['human_population']),
                       'comparison_num'] = i
    combined_pivot.loc[(combined_pivot['Group1'] == row['captive_population']) &
                       (combined_pivot['Group2'] == row['human_population']),
                       'comparison_dir'] = 'human'
    combined_pivot.loc[(combined_pivot['Group1'] == row['captive_population']) &
                       (combined_pivot['Group2'] == row['wild_population']),
                       'comparison_num'] = i
    combined_pivot.loc[(combined_pivot['Group1'] == row['captive_population']) &
                       (combined_pivot['Group2'] == row['wild_population']),
                       'comparison_dir'] = 'wild'

Remove non-target comparisons from the table

In [13]:
combined_pivot_filtered = combined_pivot.loc[(combined_pivot['comparison_num'] != 'None'),]

In [14]:
combined_pivot_filtered

Bacterial_Genus,Group1,Group2,Unassigned_____,k__Archaea_p__Crenarchaeota_c__Thaumarchaeota_o__Nitrososphaerales_f__Nitrososphaeraceae_g__CandidatusNitrososphaera,k__Archaea_p__Euryarchaeota_c__Methanobacteria_o__Methanobacteriales_f__Methanobacteriaceae_g__Methanobrevibacter,k__Archaea_p__Euryarchaeota_c__Methanobacteria_o__Methanobacteriales_f__Methanobacteriaceae_g__Methanosphaera,k__Archaea_p__Euryarchaeota_c__Thermoplasmata_o__E2_f__[Methanomassiliicoccaceae]_g__vadinCA11,k__Bacteria_____,k__Bacteria_p__Acidobacteria_c__Acidobacteria-6_o__iii1-15_f___g__,k__Bacteria_p__Acidobacteria_c__Acidobacteriia_o__Acidobacteriales_f__Acidobacteriaceae_g__,...,k__Bacteria_p__Tenericutes_c__Mollicutes_o__Mycoplasmatales_f__Mycoplasmataceae_g__,k__Bacteria_p__Tenericutes_c__Mollicutes_o__RF39_f___g__,k__Bacteria_p__Tenericutes_c__RF3_o__ML615J-28_f___g__,k__Bacteria_p__Verrucomicrobia_c__Opitutae_o__HA64_f___g__,k__Bacteria_p__Verrucomicrobia_c__Opitutae_o__[Cerasicoccales]_f__[Cerasicoccaceae]_g__,k__Bacteria_p__Verrucomicrobia_c__Verruco-5_o__WCHB1-41_f__RFP12_g__,k__Bacteria_p__Verrucomicrobia_c__Verrucomicrobiae_o__Verrucomicrobiales_f__Verrucomicrobiaceae_g__Akkermansia,k__Bacteria_p___c___o___f___g__,comparison_num,comparison_dir
26,chimp_USA_captive_campbell,chimp_DRC_wild_campbell,0.928571,NaN,NaN,0.285185,0.562698,0.969989,NaN,NaN,...,NaN,0.873083,0.855688,NaN,1.000000,1.000000,0.847937,NaN,3,wild
38,chimp_USA_captive_campbell,human_USA,NaN,NaN,1.0,NaN,0.852998,0.999667,NaN,NaN,...,NaN,0.980116,1.000000,NaN,0.929211,NaN,0.759744,NaN,3,human
40,chimp_USA_captive_moeller,chimp_TZA_wild_moeller,NaN,0.619048,1.0,0.315201,0.477241,0.952806,NaN,NaN,...,NaN,0.883538,0.871956,0.947386,1.000000,0.917548,0.960811,NaN,2,wild
51,chimp_USA_captive_moeller,human_USA,NaN,NaN,1.0,NaN,0.826263,0.998939,NaN,0.11875,...,NaN,0.980034,0.996107,NaN,0.929211,NaN,0.894608,NaN,2,human
59,douc_SGP_captive_clayton,douc_VNM_wild_clayton,NaN,NaN,NaN,NaN,NaN,0.975372,NaN,NaN,...,NaN,0.963201,0.675744,NaN,NaN,NaN,NaN,NaN,1,wild
64,douc_SGP_captive_clayton,human_USA,NaN,NaN,1.0,NaN,NaN,0.993318,NaN,0.10000,...,NaN,0.993883,1.000000,NaN,NaN,NaN,NaN,NaN,1,human
72,douc_USA_captive_clayton,douc_VNM_wild_clayton,NaN,NaN,NaN,NaN,1.000000,1.000000,NaN,NaN,...,NaN,0.998979,1.000000,NaN,NaN,NaN,NaN,NaN,0,wild
77,douc_USA_captive_clayton,human_USA,NaN,NaN,NaN,NaN,1.000000,0.981047,NaN,NaN,...,NaN,1.000000,1.000000,NaN,NaN,NaN,0.552083,NaN,0,human
85,douc_VNM_semicaptive_clayton,douc_VNM_wild_clayton,NaN,NaN,NaN,NaN,1.000000,0.873591,NaN,NaN,...,NaN,0.802844,0.679183,NaN,0.000000,NaN,NaN,NaN,6,wild
90,douc_VNM_semicaptive_clayton,human_USA,NaN,NaN,1.0,NaN,1.000000,1.000000,NaN,NaN,...,NaN,0.995633,1.000000,NaN,1.000000,NaN,NaN,NaN,6,human


## Calculate HSS

First, transpose and set column indexed on the combined filtered dataframe so that taxon is the row index and comparison number / comparison direction are a column MultiIndex

In [15]:
combined_pivot_filtered_t = combined_pivot_filtered.drop(columns=['Group1', 'Group2']).set_index(['comparison_num',
                                                                            'comparison_dir']).transpose()

In [16]:
combined_pivot_filtered_t.head()

comparison_num                                             3            \
comparison_dir                                          wild     human   
Bacterial_Genus                                                          
Unassigned_____                                     0.928571       NaN   
k__Archaea_p__Crenarchaeota_c__Thaumarchaeota_o...       NaN       NaN   
k__Archaea_p__Euryarchaeota_c__Methanobacteria_...       NaN  1.000000   
k__Archaea_p__Euryarchaeota_c__Methanobacteria_...  0.285185       NaN   
k__Archaea_p__Euryarchaeota_c__Thermoplasmata_o...  0.562698  0.852998   

comparison_num                                             2              1  \
comparison_dir                                          wild     human wild   
Bacterial_Genus                                                               
Unassigned_____                                          NaN       NaN  NaN   
k__Archaea_p__Crenarchaeota_c__Thaumarchaeota_o...  0.619048       NaN  NaN   
k__Archaea_p__Euryarchaeota_c__Methanobacteria_...  1.000000  1.000000  NaN   
k__Archaea_p__Euryarchaeota_c__Methanobacteria_...  0.315201       NaN  NaN   
k__Archaea_p__Euryarchaeota_c__Thermoplasmata_o...  0.477241  0.826263  NaN   

comparison_num                                              0          6  \
comparison_dir                                     human wild human wild   
Bacterial_Genus                                                            
Unassigned_____                                      NaN  NaN   NaN  NaN   
k__Archaea_p__Crenarchaeota_c__Thaumarchaeota_o...   NaN  NaN   NaN  NaN   
k__Archaea_p__Euryarchaeota_c__Methanobacteria_...   1.0  NaN   NaN  NaN   
k__Archaea_p__Euryarchaeota_c__Methanobacteria_...   NaN  NaN   NaN  NaN   
k__Archaea_p__Euryarchaeota_c__Thermoplasmata_o...   NaN  1.0   1.0  1.0   

comparison_num                                                   5           \
comparison_dir                                     human      wild    human   
Bacterial_Genus                                                               
Unassigned_____                                      NaN  0.907407      NaN   
k__Archaea_p__Crenarchaeota_c__Thaumarchaeota_o...   NaN       NaN      NaN   
k__Archaea_p__Euryarchaeota_c__Methanobacteria_...   1.0       NaN  1.00000   
k__Archaea_p__Euryarchaeota_c__Methanobacteria_...   NaN  0.358222      NaN   
k__Archaea_p__Euryarchaeota_c__Thermoplasmata_o...   1.0  0.796349  0.89037   

comparison_num                                        4        
comparison_dir                                     wild human  
Bacterial_Genus                                                
Unassigned_____                                     NaN   NaN  
k__Archaea_p__Crenarchaeota_c__Thaumarchaeota_o...  NaN   NaN  
k__Archaea_p__Euryarchaeota_c__Methanobacteria_...  NaN   NaN  
k__Archaea_p__Euryarchaeota_c__Methanobacteria_...  NaN   NaN  
k__Archaea_p__Euryarchaeota_c__Thermoplasmata_o...  NaN   NaN

In [17]:
target_grps = target_grp_df.copy()

In [18]:
target_grps.drop(columns=['human_population', 'wild_population'], 
                inplace=True)

In [19]:
HSS_df = pd.DataFrame(index=combined_pivot_filtered_t.index, 
                      columns=target_grps.index)

for i, row in target_grps.iterrows():
    HSS = combined_pivot_filtered_t.loc[:,(i,
                     'human')]/combined_pivot_filtered_t.loc[:,(i,
                                          'wild')]
    HSS_df[i] = HSS

In [20]:
HSS_df.head()

,0,1,2,3,4,5,6
Bacterial_Genus,,,,,,,
Unassigned_____,NaN,NaN,NaN,NaN,NaN,NaN,NaN
k__Archaea_p__Crenarchaeota_c__Thaumarchaeota_o__Nitrososphaerales_f__Nitrososphaeraceae_g__CandidatusNitrososphaera,NaN,NaN,NaN,NaN,NaN,NaN,NaN
k__Archaea_p__Euryarchaeota_c__Methanobacteria_o__Methanobacteriales_f__Methanobacteriaceae_g__Methanobrevibacter,NaN,NaN,1.000000,NaN,NaN,NaN,NaN
k__Archaea_p__Euryarchaeota_c__Methanobacteria_o__Methanobacteriales_f__Methanobacteriaceae_g__Methanosphaera,NaN,NaN,NaN,NaN,NaN,NaN,NaN
k__Archaea_p__Euryarchaeota_c__Thermoplasmata_o__E2_f__[Methanomassiliicoccaceae]_g__vadinCA11,1.0,NaN,1.731331,1.515907,NaN,1.118065,1.0


Make this pretty; add back in the comparison names

In [21]:
HSS_df_t = HSS_df.transpose()
HSS_df_t

Bacterial_Genus,Unassigned_____,k__Archaea_p__Crenarchaeota_c__Thaumarchaeota_o__Nitrososphaerales_f__Nitrososphaeraceae_g__CandidatusNitrososphaera,k__Archaea_p__Euryarchaeota_c__Methanobacteria_o__Methanobacteriales_f__Methanobacteriaceae_g__Methanobrevibacter,k__Archaea_p__Euryarchaeota_c__Methanobacteria_o__Methanobacteriales_f__Methanobacteriaceae_g__Methanosphaera,k__Archaea_p__Euryarchaeota_c__Thermoplasmata_o__E2_f__[Methanomassiliicoccaceae]_g__vadinCA11,k__Bacteria_____,k__Bacteria_p__Acidobacteria_c__Acidobacteria-6_o__iii1-15_f___g__,k__Bacteria_p__Acidobacteria_c__Acidobacteriia_o__Acidobacteriales_f__Acidobacteriaceae_g__,k__Bacteria_p__Acidobacteria_c__[Chloracidobacteria]_o__RB41_f__Ellin6075_g__,k__Bacteria_p__Actinobacteria____,...,k__Bacteria_p__Tenericutes_c__Mollicutes_o__Anaeroplasmatales_f__Anaeroplasmataceae_g__,k__Bacteria_p__Tenericutes_c__Mollicutes_o__Anaeroplasmatales_f__Anaeroplasmataceae_g__Anaeroplasma,k__Bacteria_p__Tenericutes_c__Mollicutes_o__Mycoplasmatales_f__Mycoplasmataceae_g__,k__Bacteria_p__Tenericutes_c__Mollicutes_o__RF39_f___g__,k__Bacteria_p__Tenericutes_c__RF3_o__ML615J-28_f___g__,k__Bacteria_p__Verrucomicrobia_c__Opitutae_o__HA64_f___g__,k__Bacteria_p__Verrucomicrobia_c__Opitutae_o__[Cerasicoccales]_f__[Cerasicoccaceae]_g__,k__Bacteria_p__Verrucomicrobia_c__Verruco-5_o__WCHB1-41_f__RFP12_g__,k__Bacteria_p__Verrucomicrobia_c__Verrucomicrobiae_o__Verrucomicrobiales_f__Verrucomicrobiaceae_g__Akkermansia,k__Bacteria_p___c___o___f___g__
0,NaN,NaN,NaN,NaN,1.000000,0.981047,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.001022,1.000000,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,1.018399,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.031855,1.479850,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,1.0,NaN,1.731331,1.048418,NaN,NaN,NaN,0.208661,...,0.847907,NaN,NaN,1.109215,1.142383,NaN,0.929211,NaN,0.931096,NaN
3,NaN,NaN,NaN,NaN,1.515907,1.030596,NaN,NaN,NaN,inf,...,NaN,NaN,NaN,1.122592,1.168650,NaN,0.929211,NaN,0.895992,NaN
4,NaN,NaN,NaN,NaN,NaN,1.181017,NaN,inf,NaN,0.040000,...,NaN,NaN,NaN,1.084983,NaN,NaN,5.142857,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,1.118065,1.009731,NaN,NaN,NaN,0.960584,...,NaN,NaN,NaN,1.158052,1.000000,NaN,1.011626,NaN,0.729062,NaN
6,NaN,NaN,NaN,NaN,1.000000,1.144701,NaN,NaN,NaN,NaN,...,2.139109,NaN,NaN,1.240132,1.472357,NaN,inf,NaN,NaN,NaN


In [22]:
target_grp_df.columns

Index(['captive_population', 'human_population', 'wild_population'], dtype='object')

In [23]:
final_HSS_table = HSS_df_t.merge(target_grp_df,
                      left_index=True,
                      right_index=True).set_index(list(target_grp_df.columns)).transpose()

In [24]:
final_HSS_table.head()

captive_population,douc_USA_captive_clayton,douc_SGP_captive_clayton,chimp_USA_captive_moeller,chimp_USA_captive_campbell,howler_CRI_captive_clayton,gorilla_USA_captive_campbell,douc_VNM_semicaptive_clayton
human_population,human_USA,human_USA,human_USA,human_USA,human_USA,human_USA,human_USA
wild_population,douc_VNM_wild_clayton,douc_VNM_wild_clayton,chimp_TZA_wild_moeller,chimp_DRC_wild_campbell,howler_CRI_wild_clayton,gorilla_DRC_wild_campbell,douc_VNM_wild_clayton
Unassigned_____,NaN,NaN,NaN,NaN,NaN,NaN,NaN
k__Archaea_p__Crenarchaeota_c__Thaumarchaeota_o__Nitrososphaerales_f__Nitrososphaeraceae_g__CandidatusNitrososphaera,NaN,NaN,NaN,NaN,NaN,NaN,NaN
k__Archaea_p__Euryarchaeota_c__Methanobacteria_o__Methanobacteriales_f__Methanobacteriaceae_g__Methanobrevibacter,NaN,NaN,1.000000,NaN,NaN,NaN,NaN
k__Archaea_p__Euryarchaeota_c__Methanobacteria_o__Methanobacteriales_f__Methanobacteriaceae_g__Methanosphaera,NaN,NaN,NaN,NaN,NaN,NaN,NaN
k__Archaea_p__Euryarchaeota_c__Thermoplasmata_o__E2_f__[Methanomassiliicoccaceae]_g__vadinCA11,1.0,NaN,1.731331,1.515907,NaN,1.118065,1.0


Write to file

In [25]:
outdir = join('output', 'results')
makedirs(outdir, exist_ok=True)
final_HSS_table.to_csv(join(outdir, 'hss-yatsunenko.txt'), sep='\t')

## Calculate Microbiota Convergence Score

The MCS is the ratio of [captive vs human] / [wild vs human]

## Filter data list to target comparisons

Update the aggregated pivot table to include the comparison number and comparison direction (captive to human or human to wild) as additional columns

In [26]:
combined_pivot['comparison_num'] = 'None'
combined_pivot['comparison_dir'] = 'None'

In [27]:
combined_pivot.head()

Bacterial_Genus,Group1,Group2,Unassigned_____,k__Archaea_p__Crenarchaeota_c__Thaumarchaeota_o__Nitrososphaerales_f__Nitrososphaeraceae_g__CandidatusNitrososphaera,k__Archaea_p__Euryarchaeota_c__Methanobacteria_o__Methanobacteriales_f__Methanobacteriaceae_g__Methanobrevibacter,k__Archaea_p__Euryarchaeota_c__Methanobacteria_o__Methanobacteriales_f__Methanobacteriaceae_g__Methanosphaera,k__Archaea_p__Euryarchaeota_c__Thermoplasmata_o__E2_f__[Methanomassiliicoccaceae]_g__vadinCA11,k__Bacteria_____,k__Bacteria_p__Acidobacteria_c__Acidobacteria-6_o__iii1-15_f___g__,k__Bacteria_p__Acidobacteria_c__Acidobacteriia_o__Acidobacteriales_f__Acidobacteriaceae_g__,...,k__Bacteria_p__Tenericutes_c__Mollicutes_o__Mycoplasmatales_f__Mycoplasmataceae_g__,k__Bacteria_p__Tenericutes_c__Mollicutes_o__RF39_f___g__,k__Bacteria_p__Tenericutes_c__RF3_o__ML615J-28_f___g__,k__Bacteria_p__Verrucomicrobia_c__Opitutae_o__HA64_f___g__,k__Bacteria_p__Verrucomicrobia_c__Opitutae_o__[Cerasicoccales]_f__[Cerasicoccaceae]_g__,k__Bacteria_p__Verrucomicrobia_c__Verruco-5_o__WCHB1-41_f__RFP12_g__,k__Bacteria_p__Verrucomicrobia_c__Verrucomicrobiae_o__Verrucomicrobiales_f__Verrucomicrobiaceae_g__Akkermansia,k__Bacteria_p___c___o___f___g__,comparison_num,comparison_dir
0,chimp_DRC_wild_campbell,chimp_DRC_wild_campbell,0.920635,NaN,NaN,NaN,0.200000,0.846254,0.333333,1.000000,...,NaN,0.632296,1.000000,NaN,0.135761,0.646717,0.553968,1.000000,None,None
1,chimp_DRC_wild_campbell,chimp_TZA_wild_moeller,1.000000,NaN,NaN,0.197619,0.349583,0.962346,0.852381,NaN,...,NaN,0.795701,0.784660,NaN,0.430928,0.705881,0.405233,0.821429,None,None
2,chimp_DRC_wild_campbell,chimp_USA_captive_campbell,0.928571,NaN,NaN,0.285185,0.562698,0.969989,NaN,NaN,...,NaN,0.873083,0.855688,NaN,1.000000,1.000000,0.847937,NaN,None,None
3,chimp_DRC_wild_campbell,chimp_USA_captive_moeller,NaN,NaN,NaN,0.217949,0.561645,0.983341,NaN,0.510417,...,NaN,0.895034,0.896667,NaN,1.000000,1.000000,0.839608,NaN,None,None
4,chimp_DRC_wild_campbell,douc_SGP_captive_clayton,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,0.500000,...,NaN,0.991665,1.000000,NaN,NaN,1.000000,NaN,NaN,None,None


In [28]:
target_grp_df

,captive_population,human_population,wild_population
0,douc_USA_captive_clayton,human_USA,douc_VNM_wild_clayton
1,douc_SGP_captive_clayton,human_USA,douc_VNM_wild_clayton
2,chimp_USA_captive_moeller,human_USA,chimp_TZA_wild_moeller
3,chimp_USA_captive_campbell,human_USA,chimp_DRC_wild_campbell
4,howler_CRI_captive_clayton,human_USA,howler_CRI_wild_clayton
5,gorilla_USA_captive_campbell,human_USA,gorilla_DRC_wild_campbell
6,douc_VNM_semicaptive_clayton,human_USA,douc_VNM_wild_clayton


In [29]:
for i, row in target_grp_df.iterrows():
    combined_pivot.loc[(combined_pivot['Group1'] == row['captive_population']) &
                       (combined_pivot['Group2'] == row['human_population']),
                       'comparison_num'] = i
    combined_pivot.loc[(combined_pivot['Group1'] == row['captive_population']) &
                       (combined_pivot['Group2'] == row['human_population']),
                       'comparison_dir'] = 'captive-human'
    combined_pivot.loc[(combined_pivot['Group1'] == row['wild_population']) &
                       (combined_pivot['Group2'] == row['human_population']),
                       'comparison_num'] = i
    combined_pivot.loc[(combined_pivot['Group1'] == row['wild_population']) &
                       (combined_pivot['Group2'] == row['human_population']),
                       'comparison_dir'] = 'wild-human'

In [30]:
combined_pivot.loc[(combined_pivot['comparison_num'] != 'None'),]

Bacterial_Genus,Group1,Group2,Unassigned_____,k__Archaea_p__Crenarchaeota_c__Thaumarchaeota_o__Nitrososphaerales_f__Nitrososphaeraceae_g__CandidatusNitrososphaera,k__Archaea_p__Euryarchaeota_c__Methanobacteria_o__Methanobacteriales_f__Methanobacteriaceae_g__Methanobrevibacter,k__Archaea_p__Euryarchaeota_c__Methanobacteria_o__Methanobacteriales_f__Methanobacteriaceae_g__Methanosphaera,k__Archaea_p__Euryarchaeota_c__Thermoplasmata_o__E2_f__[Methanomassiliicoccaceae]_g__vadinCA11,k__Bacteria_____,k__Bacteria_p__Acidobacteria_c__Acidobacteria-6_o__iii1-15_f___g__,k__Bacteria_p__Acidobacteria_c__Acidobacteriia_o__Acidobacteriales_f__Acidobacteriaceae_g__,...,k__Bacteria_p__Tenericutes_c__Mollicutes_o__Mycoplasmatales_f__Mycoplasmataceae_g__,k__Bacteria_p__Tenericutes_c__Mollicutes_o__RF39_f___g__,k__Bacteria_p__Tenericutes_c__RF3_o__ML615J-28_f___g__,k__Bacteria_p__Verrucomicrobia_c__Opitutae_o__HA64_f___g__,k__Bacteria_p__Verrucomicrobia_c__Opitutae_o__[Cerasicoccales]_f__[Cerasicoccaceae]_g__,k__Bacteria_p__Verrucomicrobia_c__Verruco-5_o__WCHB1-41_f__RFP12_g__,k__Bacteria_p__Verrucomicrobia_c__Verrucomicrobiae_o__Verrucomicrobiales_f__Verrucomicrobiaceae_g__Akkermansia,k__Bacteria_p___c___o___f___g__,comparison_num,comparison_dir
12,chimp_DRC_wild_campbell,human_USA,NaN,NaN,NaN,NaN,0.711111,0.997188,NaN,0.55000,...,NaN,0.977664,1.000000,NaN,1.000000,NaN,0.948148,NaN,3,wild-human
25,chimp_TZA_wild_moeller,human_USA,NaN,NaN,1.0,NaN,0.821065,0.996487,NaN,NaN,...,NaN,0.983282,0.990870,NaN,1.000000,NaN,1.000000,NaN,2,wild-human
38,chimp_USA_captive_campbell,human_USA,NaN,NaN,1.0,NaN,0.852998,0.999667,NaN,NaN,...,NaN,0.980116,1.000000,NaN,0.929211,NaN,0.759744,NaN,3,captive-human
51,chimp_USA_captive_moeller,human_USA,NaN,NaN,1.0,NaN,0.826263,0.998939,NaN,0.11875,...,NaN,0.980034,0.996107,NaN,0.929211,NaN,0.894608,NaN,2,captive-human
64,douc_SGP_captive_clayton,human_USA,NaN,NaN,1.0,NaN,NaN,0.993318,NaN,0.10000,...,NaN,0.993883,1.000000,NaN,NaN,NaN,NaN,NaN,1,captive-human
77,douc_USA_captive_clayton,human_USA,NaN,NaN,NaN,NaN,1.000000,0.981047,NaN,NaN,...,NaN,1.000000,1.000000,NaN,NaN,NaN,0.552083,NaN,0,captive-human
90,douc_VNM_semicaptive_clayton,human_USA,NaN,NaN,1.0,NaN,1.000000,1.000000,NaN,NaN,...,NaN,0.995633,1.000000,NaN,1.000000,NaN,NaN,NaN,6,captive-human
103,douc_VNM_wild_clayton,human_USA,NaN,NaN,NaN,NaN,1.000000,1.000000,NaN,NaN,...,NaN,0.993552,1.000000,NaN,1.000000,NaN,NaN,NaN,6,wild-human
116,gorilla_DRC_wild_campbell,human_USA,NaN,NaN,NaN,NaN,1.000000,0.999783,NaN,1.00000,...,NaN,0.985904,1.000000,NaN,0.997094,NaN,0.968137,NaN,5,wild-human
129,gorilla_USA_captive_campbell,human_USA,NaN,NaN,1.0,NaN,0.890370,0.999330,NaN,NaN,...,NaN,0.985929,1.000000,NaN,0.929331,NaN,0.628571,NaN,5,captive-human


Remove non-target comparisons from the table

In [31]:
combined_pivot_filtered_MCS = combined_pivot.loc[(combined_pivot['comparison_num'] != 'None'),]

In [32]:
combined_pivot_filtered_MCS

Bacterial_Genus,Group1,Group2,Unassigned_____,k__Archaea_p__Crenarchaeota_c__Thaumarchaeota_o__Nitrososphaerales_f__Nitrososphaeraceae_g__CandidatusNitrososphaera,k__Archaea_p__Euryarchaeota_c__Methanobacteria_o__Methanobacteriales_f__Methanobacteriaceae_g__Methanobrevibacter,k__Archaea_p__Euryarchaeota_c__Methanobacteria_o__Methanobacteriales_f__Methanobacteriaceae_g__Methanosphaera,k__Archaea_p__Euryarchaeota_c__Thermoplasmata_o__E2_f__[Methanomassiliicoccaceae]_g__vadinCA11,k__Bacteria_____,k__Bacteria_p__Acidobacteria_c__Acidobacteria-6_o__iii1-15_f___g__,k__Bacteria_p__Acidobacteria_c__Acidobacteriia_o__Acidobacteriales_f__Acidobacteriaceae_g__,...,k__Bacteria_p__Tenericutes_c__Mollicutes_o__Mycoplasmatales_f__Mycoplasmataceae_g__,k__Bacteria_p__Tenericutes_c__Mollicutes_o__RF39_f___g__,k__Bacteria_p__Tenericutes_c__RF3_o__ML615J-28_f___g__,k__Bacteria_p__Verrucomicrobia_c__Opitutae_o__HA64_f___g__,k__Bacteria_p__Verrucomicrobia_c__Opitutae_o__[Cerasicoccales]_f__[Cerasicoccaceae]_g__,k__Bacteria_p__Verrucomicrobia_c__Verruco-5_o__WCHB1-41_f__RFP12_g__,k__Bacteria_p__Verrucomicrobia_c__Verrucomicrobiae_o__Verrucomicrobiales_f__Verrucomicrobiaceae_g__Akkermansia,k__Bacteria_p___c___o___f___g__,comparison_num,comparison_dir
12,chimp_DRC_wild_campbell,human_USA,NaN,NaN,NaN,NaN,0.711111,0.997188,NaN,0.55000,...,NaN,0.977664,1.000000,NaN,1.000000,NaN,0.948148,NaN,3,wild-human
25,chimp_TZA_wild_moeller,human_USA,NaN,NaN,1.0,NaN,0.821065,0.996487,NaN,NaN,...,NaN,0.983282,0.990870,NaN,1.000000,NaN,1.000000,NaN,2,wild-human
38,chimp_USA_captive_campbell,human_USA,NaN,NaN,1.0,NaN,0.852998,0.999667,NaN,NaN,...,NaN,0.980116,1.000000,NaN,0.929211,NaN,0.759744,NaN,3,captive-human
51,chimp_USA_captive_moeller,human_USA,NaN,NaN,1.0,NaN,0.826263,0.998939,NaN,0.11875,...,NaN,0.980034,0.996107,NaN,0.929211,NaN,0.894608,NaN,2,captive-human
64,douc_SGP_captive_clayton,human_USA,NaN,NaN,1.0,NaN,NaN,0.993318,NaN,0.10000,...,NaN,0.993883,1.000000,NaN,NaN,NaN,NaN,NaN,1,captive-human
77,douc_USA_captive_clayton,human_USA,NaN,NaN,NaN,NaN,1.000000,0.981047,NaN,NaN,...,NaN,1.000000,1.000000,NaN,NaN,NaN,0.552083,NaN,0,captive-human
90,douc_VNM_semicaptive_clayton,human_USA,NaN,NaN,1.0,NaN,1.000000,1.000000,NaN,NaN,...,NaN,0.995633,1.000000,NaN,1.000000,NaN,NaN,NaN,6,captive-human
103,douc_VNM_wild_clayton,human_USA,NaN,NaN,NaN,NaN,1.000000,1.000000,NaN,NaN,...,NaN,0.993552,1.000000,NaN,1.000000,NaN,NaN,NaN,6,wild-human
116,gorilla_DRC_wild_campbell,human_USA,NaN,NaN,NaN,NaN,1.000000,0.999783,NaN,1.00000,...,NaN,0.985904,1.000000,NaN,0.997094,NaN,0.968137,NaN,5,wild-human
129,gorilla_USA_captive_campbell,human_USA,NaN,NaN,1.0,NaN,0.890370,0.999330,NaN,NaN,...,NaN,0.985929,1.000000,NaN,0.929331,NaN,0.628571,NaN,5,captive-human


## Calculate MCS

First, transpose and set column indexed on the combined filtered dataframe so that taxon is the row index and comparison number / comparison direction are a column MultiIndex

In [33]:
combined_pivot_filtered_MCS

Bacterial_Genus,Group1,Group2,Unassigned_____,k__Archaea_p__Crenarchaeota_c__Thaumarchaeota_o__Nitrososphaerales_f__Nitrososphaeraceae_g__CandidatusNitrososphaera,k__Archaea_p__Euryarchaeota_c__Methanobacteria_o__Methanobacteriales_f__Methanobacteriaceae_g__Methanobrevibacter,k__Archaea_p__Euryarchaeota_c__Methanobacteria_o__Methanobacteriales_f__Methanobacteriaceae_g__Methanosphaera,k__Archaea_p__Euryarchaeota_c__Thermoplasmata_o__E2_f__[Methanomassiliicoccaceae]_g__vadinCA11,k__Bacteria_____,k__Bacteria_p__Acidobacteria_c__Acidobacteria-6_o__iii1-15_f___g__,k__Bacteria_p__Acidobacteria_c__Acidobacteriia_o__Acidobacteriales_f__Acidobacteriaceae_g__,...,k__Bacteria_p__Tenericutes_c__Mollicutes_o__Mycoplasmatales_f__Mycoplasmataceae_g__,k__Bacteria_p__Tenericutes_c__Mollicutes_o__RF39_f___g__,k__Bacteria_p__Tenericutes_c__RF3_o__ML615J-28_f___g__,k__Bacteria_p__Verrucomicrobia_c__Opitutae_o__HA64_f___g__,k__Bacteria_p__Verrucomicrobia_c__Opitutae_o__[Cerasicoccales]_f__[Cerasicoccaceae]_g__,k__Bacteria_p__Verrucomicrobia_c__Verruco-5_o__WCHB1-41_f__RFP12_g__,k__Bacteria_p__Verrucomicrobia_c__Verrucomicrobiae_o__Verrucomicrobiales_f__Verrucomicrobiaceae_g__Akkermansia,k__Bacteria_p___c___o___f___g__,comparison_num,comparison_dir
12,chimp_DRC_wild_campbell,human_USA,NaN,NaN,NaN,NaN,0.711111,0.997188,NaN,0.55000,...,NaN,0.977664,1.000000,NaN,1.000000,NaN,0.948148,NaN,3,wild-human
25,chimp_TZA_wild_moeller,human_USA,NaN,NaN,1.0,NaN,0.821065,0.996487,NaN,NaN,...,NaN,0.983282,0.990870,NaN,1.000000,NaN,1.000000,NaN,2,wild-human
38,chimp_USA_captive_campbell,human_USA,NaN,NaN,1.0,NaN,0.852998,0.999667,NaN,NaN,...,NaN,0.980116,1.000000,NaN,0.929211,NaN,0.759744,NaN,3,captive-human
51,chimp_USA_captive_moeller,human_USA,NaN,NaN,1.0,NaN,0.826263,0.998939,NaN,0.11875,...,NaN,0.980034,0.996107,NaN,0.929211,NaN,0.894608,NaN,2,captive-human
64,douc_SGP_captive_clayton,human_USA,NaN,NaN,1.0,NaN,NaN,0.993318,NaN,0.10000,...,NaN,0.993883,1.000000,NaN,NaN,NaN,NaN,NaN,1,captive-human
77,douc_USA_captive_clayton,human_USA,NaN,NaN,NaN,NaN,1.000000,0.981047,NaN,NaN,...,NaN,1.000000,1.000000,NaN,NaN,NaN,0.552083,NaN,0,captive-human
90,douc_VNM_semicaptive_clayton,human_USA,NaN,NaN,1.0,NaN,1.000000,1.000000,NaN,NaN,...,NaN,0.995633,1.000000,NaN,1.000000,NaN,NaN,NaN,6,captive-human
103,douc_VNM_wild_clayton,human_USA,NaN,NaN,NaN,NaN,1.000000,1.000000,NaN,NaN,...,NaN,0.993552,1.000000,NaN,1.000000,NaN,NaN,NaN,6,wild-human
116,gorilla_DRC_wild_campbell,human_USA,NaN,NaN,NaN,NaN,1.000000,0.999783,NaN,1.00000,...,NaN,0.985904,1.000000,NaN,0.997094,NaN,0.968137,NaN,5,wild-human
129,gorilla_USA_captive_campbell,human_USA,NaN,NaN,1.0,NaN,0.890370,0.999330,NaN,NaN,...,NaN,0.985929,1.000000,NaN,0.929331,NaN,0.628571,NaN,5,captive-human


Because we are re-using the Douc-Wild comparisons, we need to manually replicate those rows in the data frame.

In [38]:
douc_0_wild = combined_pivot_filtered_MCS.loc[
        (combined_pivot_filtered_MCS['comparison_num'] == 6) &
        (combined_pivot_filtered_MCS['comparison_dir'] == 'wild-human')].reset_index(drop=True)

In [39]:
douc_0_wild

Bacterial_Genus,Group1,Group2,Unassigned_____,k__Archaea_p__Crenarchaeota_c__Thaumarchaeota_o__Nitrososphaerales_f__Nitrososphaeraceae_g__CandidatusNitrososphaera,k__Archaea_p__Euryarchaeota_c__Methanobacteria_o__Methanobacteriales_f__Methanobacteriaceae_g__Methanobrevibacter,k__Archaea_p__Euryarchaeota_c__Methanobacteria_o__Methanobacteriales_f__Methanobacteriaceae_g__Methanosphaera,k__Archaea_p__Euryarchaeota_c__Thermoplasmata_o__E2_f__[Methanomassiliicoccaceae]_g__vadinCA11,k__Bacteria_____,k__Bacteria_p__Acidobacteria_c__Acidobacteria-6_o__iii1-15_f___g__,k__Bacteria_p__Acidobacteria_c__Acidobacteriia_o__Acidobacteriales_f__Acidobacteriaceae_g__,...,k__Bacteria_p__Tenericutes_c__Mollicutes_o__Mycoplasmatales_f__Mycoplasmataceae_g__,k__Bacteria_p__Tenericutes_c__Mollicutes_o__RF39_f___g__,k__Bacteria_p__Tenericutes_c__RF3_o__ML615J-28_f___g__,k__Bacteria_p__Verrucomicrobia_c__Opitutae_o__HA64_f___g__,k__Bacteria_p__Verrucomicrobia_c__Opitutae_o__[Cerasicoccales]_f__[Cerasicoccaceae]_g__,k__Bacteria_p__Verrucomicrobia_c__Verruco-5_o__WCHB1-41_f__RFP12_g__,k__Bacteria_p__Verrucomicrobia_c__Verrucomicrobiae_o__Verrucomicrobiales_f__Verrucomicrobiaceae_g__Akkermansia,k__Bacteria_p___c___o___f___g__,comparison_num,comparison_dir
0,douc_VNM_wild_clayton,human_USA,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,...,NaN,0.993552,1.0,NaN,1.0,NaN,NaN,NaN,6,wild-human


In [40]:
douc_0_wild.loc[douc_0_wild.index[0],
                'comparison_num'] = 0

In [41]:
douc_1_wild = combined_pivot_filtered_MCS.loc[
        (combined_pivot_filtered_MCS['comparison_num'] == 6) &
        (combined_pivot_filtered_MCS['comparison_dir'] == 'wild-human')].reset_index(drop=True)

In [42]:
douc_1_wild.loc[douc_0_wild.index[0],
                'comparison_num'] = 1

In [43]:
combined_pivot_filtered_MCS = combined_pivot_filtered_MCS.append([douc_0_wild, douc_1_wild]).reset_index(drop=True)

In [44]:
combined_pivot_filtered_MCS_t = combined_pivot_filtered_MCS.drop(columns=['Group1', 'Group2']).set_index(['comparison_num',
                                                                            'comparison_dir']).transpose().sort_index(axis=1)

In [45]:
combined_pivot_filtered_MCS_t.head()

comparison_num                                                 0             \
comparison_dir                                     captive-human wild-human   
Bacterial_Genus                                                               
Unassigned_____                                              NaN        NaN   
k__Archaea_p__Crenarchaeota_c__Thaumarchaeota_o...           NaN        NaN   
k__Archaea_p__Euryarchaeota_c__Methanobacteria_...           NaN        NaN   
k__Archaea_p__Euryarchaeota_c__Methanobacteria_...           NaN        NaN   
k__Archaea_p__Euryarchaeota_c__Thermoplasmata_o...           1.0        1.0   

comparison_num                                                 1             \
comparison_dir                                     captive-human wild-human   
Bacterial_Genus                                                               
Unassigned_____                                              NaN        NaN   
k__Archaea_p__Crenarchaeota_c__Thaumarchaeota_o...           NaN        NaN   
k__Archaea_p__Euryarchaeota_c__Methanobacteria_...           1.0        NaN   
k__Archaea_p__Euryarchaeota_c__Methanobacteria_...           NaN        NaN   
k__Archaea_p__Euryarchaeota_c__Thermoplasmata_o...           NaN        1.0   

comparison_num                                                 2             \
comparison_dir                                     captive-human wild-human   
Bacterial_Genus                                                               
Unassigned_____                                              NaN        NaN   
k__Archaea_p__Crenarchaeota_c__Thaumarchaeota_o...           NaN        NaN   
k__Archaea_p__Euryarchaeota_c__Methanobacteria_...      1.000000   1.000000   
k__Archaea_p__Euryarchaeota_c__Methanobacteria_...           NaN        NaN   
k__Archaea_p__Euryarchaeota_c__Thermoplasmata_o...      0.826263   0.821065   

comparison_num                                                 3             \
comparison_dir                                     captive-human wild-human   
Bacterial_Genus                                                               
Unassigned_____                                              NaN        NaN   
k__Archaea_p__Crenarchaeota_c__Thaumarchaeota_o...           NaN        NaN   
k__Archaea_p__Euryarchaeota_c__Methanobacteria_...      1.000000        NaN   
k__Archaea_p__Euryarchaeota_c__Methanobacteria_...           NaN        NaN   
k__Archaea_p__Euryarchaeota_c__Thermoplasmata_o...      0.852998   0.711111   

comparison_num                                                 4             \
comparison_dir                                     captive-human wild-human   
Bacterial_Genus                                                               
Unassigned_____                                              NaN        NaN   
k__Archaea_p__Crenarchaeota_c__Thaumarchaeota_o...           NaN        NaN   
k__Archaea_p__Euryarchaeota_c__Methanobacteria_...           NaN        NaN   
k__Archaea_p__Euryarchaeota_c__Methanobacteria_...           NaN        NaN   
k__Archaea_p__Euryarchaeota_c__Thermoplasmata_o...           NaN        NaN   

comparison_num                                                 5             \
comparison_dir                                     captive-human wild-human   
Bacterial_Genus                                                               
Unassigned_____                                              NaN        NaN   
k__Archaea_p__Crenarchaeota_c__Thaumarchaeota_o...           NaN        NaN   
k__Archaea_p__Euryarchaeota_c__Methanobacteria_...       1.00000        NaN   
k__Archaea_p__Euryarchaeota_c__Methanobacteria_...           NaN        NaN   
k__Archaea_p__Euryarchaeota_c__Thermoplasmata_o...       0.89037        1.0   

comparison_num                                                 6             
comparison_dir                                     captive-human wild-human  
Bacterial_Genus                               

In [46]:
target_grps = target_grp_df.copy()

In [47]:
target_grps.drop(columns=['human_population', 'wild_population'], 
                inplace=True)

In [48]:
target_grps

,captive_population
0,douc_USA_captive_clayton
1,douc_SGP_captive_clayton
2,chimp_USA_captive_moeller
3,chimp_USA_captive_campbell
4,howler_CRI_captive_clayton
5,gorilla_USA_captive_campbell
6,douc_VNM_semicaptive_clayton


In [50]:
MCS_df = pd.DataFrame(index=combined_pivot_filtered_MCS_t.index, 
                      columns=target_grps.index)

for i, row in target_grps.iterrows():
    print(row)
    print(i)
    MCS = combined_pivot_filtered_MCS_t.loc[:,(i,
                     'captive-human')]/combined_pivot_filtered_MCS_t.loc[:,(i,
                                          'wild-human')]
    MCS_df[i] = MCS

captive_population    douc_USA_captive_clayton
Name: 0, dtype: object
0
captive_population    douc_SGP_captive_clayton
Name: 1, dtype: object
1
captive_population    chimp_USA_captive_moeller
Name: 2, dtype: object
2
captive_population    chimp_USA_captive_campbell
Name: 3, dtype: object
3
captive_population    howler_CRI_captive_clayton
Name: 4, dtype: object
4
captive_population    gorilla_USA_captive_campbell
Name: 5, dtype: object
5
captive_population    douc_VNM_semicaptive_clayton
Name: 6, dtype: object
6


In [51]:
MCS_df.head()

,0,1,2,3,4,5,6
Bacterial_Genus,,,,,,,
Unassigned_____,NaN,NaN,NaN,NaN,NaN,NaN,NaN
k__Archaea_p__Crenarchaeota_c__Thaumarchaeota_o__Nitrososphaerales_f__Nitrososphaeraceae_g__CandidatusNitrososphaera,NaN,NaN,NaN,NaN,NaN,NaN,NaN
k__Archaea_p__Euryarchaeota_c__Methanobacteria_o__Methanobacteriales_f__Methanobacteriaceae_g__Methanobrevibacter,NaN,NaN,1.000000,NaN,NaN,NaN,NaN
k__Archaea_p__Euryarchaeota_c__Methanobacteria_o__Methanobacteriales_f__Methanobacteriaceae_g__Methanosphaera,NaN,NaN,NaN,NaN,NaN,NaN,NaN
k__Archaea_p__Euryarchaeota_c__Thermoplasmata_o__E2_f__[Methanomassiliicoccaceae]_g__vadinCA11,1.0,NaN,1.006331,1.199529,NaN,0.89037,1.0


Make this pretty; add back in the comparison names

In [52]:
MCS_df_t = MCS_df.transpose()
MCS_df_t

Bacterial_Genus,Unassigned_____,k__Archaea_p__Crenarchaeota_c__Thaumarchaeota_o__Nitrososphaerales_f__Nitrososphaeraceae_g__CandidatusNitrososphaera,k__Archaea_p__Euryarchaeota_c__Methanobacteria_o__Methanobacteriales_f__Methanobacteriaceae_g__Methanobrevibacter,k__Archaea_p__Euryarchaeota_c__Methanobacteria_o__Methanobacteriales_f__Methanobacteriaceae_g__Methanosphaera,k__Archaea_p__Euryarchaeota_c__Thermoplasmata_o__E2_f__[Methanomassiliicoccaceae]_g__vadinCA11,k__Bacteria_____,k__Bacteria_p__Acidobacteria_c__Acidobacteria-6_o__iii1-15_f___g__,k__Bacteria_p__Acidobacteria_c__Acidobacteriia_o__Acidobacteriales_f__Acidobacteriaceae_g__,k__Bacteria_p__Acidobacteria_c__[Chloracidobacteria]_o__RB41_f__Ellin6075_g__,k__Bacteria_p__Actinobacteria____,...,k__Bacteria_p__Tenericutes_c__Mollicutes_o__Anaeroplasmatales_f__Anaeroplasmataceae_g__,k__Bacteria_p__Tenericutes_c__Mollicutes_o__Anaeroplasmatales_f__Anaeroplasmataceae_g__Anaeroplasma,k__Bacteria_p__Tenericutes_c__Mollicutes_o__Mycoplasmatales_f__Mycoplasmataceae_g__,k__Bacteria_p__Tenericutes_c__Mollicutes_o__RF39_f___g__,k__Bacteria_p__Tenericutes_c__RF3_o__ML615J-28_f___g__,k__Bacteria_p__Verrucomicrobia_c__Opitutae_o__HA64_f___g__,k__Bacteria_p__Verrucomicrobia_c__Opitutae_o__[Cerasicoccales]_f__[Cerasicoccaceae]_g__,k__Bacteria_p__Verrucomicrobia_c__Verruco-5_o__WCHB1-41_f__RFP12_g__,k__Bacteria_p__Verrucomicrobia_c__Verrucomicrobiae_o__Verrucomicrobiales_f__Verrucomicrobiaceae_g__Akkermansia,k__Bacteria_p___c___o___f___g__
0,NaN,NaN,NaN,NaN,1.000000,0.981047,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.006489,1.000000,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,0.993318,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.000333,1.000000,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,1.0,NaN,1.006331,1.002461,NaN,NaN,NaN,0.201355,...,0.744186,NaN,NaN,0.996696,1.005285,NaN,0.929211,NaN,0.894608,NaN
3,NaN,NaN,NaN,NaN,1.199529,1.002486,NaN,NaN,NaN,1.000000,...,NaN,NaN,NaN,1.002508,1.000000,NaN,0.929211,NaN,0.801293,NaN
4,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,1.0,NaN,0.039735,...,NaN,NaN,NaN,1.003621,NaN,NaN,1.000000,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,0.890370,0.999547,NaN,NaN,NaN,0.831858,...,NaN,NaN,NaN,1.000026,1.000000,NaN,0.932039,NaN,0.649259,NaN
6,NaN,NaN,NaN,NaN,1.000000,1.000000,NaN,NaN,NaN,NaN,...,1.000000,NaN,NaN,1.002094,1.000000,NaN,1.000000,NaN,NaN,NaN


In [53]:
target_grp_df.columns

Index(['captive_population', 'human_population', 'wild_population'], dtype='object')

In [54]:
final_MCS_table = MCS_df_t.merge(target_grp_df,
                      left_index=True,
                      right_index=True).set_index(list(target_grp_df.columns)).transpose()

In [55]:
final_MCS_table.head()

captive_population,douc_USA_captive_clayton,douc_SGP_captive_clayton,chimp_USA_captive_moeller,chimp_USA_captive_campbell,howler_CRI_captive_clayton,gorilla_USA_captive_campbell,douc_VNM_semicaptive_clayton
human_population,human_USA,human_USA,human_USA,human_USA,human_USA,human_USA,human_USA
wild_population,douc_VNM_wild_clayton,douc_VNM_wild_clayton,chimp_TZA_wild_moeller,chimp_DRC_wild_campbell,howler_CRI_wild_clayton,gorilla_DRC_wild_campbell,douc_VNM_wild_clayton
Unassigned_____,NaN,NaN,NaN,NaN,NaN,NaN,NaN
k__Archaea_p__Crenarchaeota_c__Thaumarchaeota_o__Nitrososphaerales_f__Nitrososphaeraceae_g__CandidatusNitrososphaera,NaN,NaN,NaN,NaN,NaN,NaN,NaN
k__Archaea_p__Euryarchaeota_c__Methanobacteria_o__Methanobacteriales_f__Methanobacteriaceae_g__Methanobrevibacter,NaN,NaN,1.000000,NaN,NaN,NaN,NaN
k__Archaea_p__Euryarchaeota_c__Methanobacteria_o__Methanobacteriales_f__Methanobacteriaceae_g__Methanosphaera,NaN,NaN,NaN,NaN,NaN,NaN,NaN
k__Archaea_p__Euryarchaeota_c__Thermoplasmata_o__E2_f__[Methanomassiliicoccaceae]_g__vadinCA11,1.0,NaN,1.006331,1.199529,NaN,0.89037,1.0


Write to file

In [56]:
outdir = join('output', 'results')
makedirs(outdir, exist_ok=True)
final_MCS_table.to_csv(join(outdir, 'mcs-yatsunenko.txt'), sep='\t')

# Data set 2: American Gut

In [83]:
group_sig_out_dir = './output/distance-amgut'

## Import data

Get a list of all the files in the output group significance directory

In [84]:
taxa = listdir(group_sig_out_dir)

Iterate over each of the output files, and import the group significance results

In [85]:
# grab tsv files
bact_dfs = []
for d in taxa:
    data_fp = join(group_sig_out_dir,
                   d,
                   'permanova',
                   'raw_data.tsv')
    
    d_df = pd.read_csv(data_fp, 
                       header=0,
                       sep='\t',
                       index_col=0)

    d_df['Bacterial_Genus'] = d
    
    bact_dfs.append(d_df)
    

# load group significance tsvs


Combine these results into a single dataframe.

In [86]:
combined_df = pd.concat(bact_dfs, ignore_index=True)

In [87]:
combined_df.head()

,SubjectID1,SubjectID2,Group1,Group2,Distance,Bacterial_Genus
0,100030.13393.SRR8978339,100030.13393.SRR8978285,chimp_DRC_wild_campbell,chimp_DRC_wild_campbell,0.0,k__Bacteria_p___c___o___f___g__
1,100030.13393.SRR8978285,65716.12173.14,chimp_DRC_wild_campbell,chimp_TZA_wild_moeller,0.0,k__Bacteria_p___c___o___f___g__
2,100030.13393.SRR8978285,65716.12173.35,chimp_DRC_wild_campbell,chimp_TZA_wild_moeller,0.0,k__Bacteria_p___c___o___f___g__
3,100030.13393.SRR8978285,65716.12173.135,chimp_DRC_wild_campbell,chimp_TZA_wild_moeller,0.0,k__Bacteria_p___c___o___f___g__
4,100030.13393.SRR8978285,65716.12173.26,chimp_DRC_wild_campbell,chimp_TZA_wild_moeller,0.0,k__Bacteria_p___c___o___f___g__


## Aggregate distance values

Using the Pandas pivot table functionality, aggregate the individual dissimilarity comparisons by group using `np.mean`.

In [88]:
combined_pivot = combined_df.pivot_table(values='Distance',
                        index=['Group1', 'Group2'],
                        columns=['Bacterial_Genus'],
                        aggfunc=np.mean)

In [89]:
combined_pivot.reset_index(inplace=True)

In [90]:
combined_pivot.head()


Bacterial_Genus,Group1,Group2,Unassigned_____,k__Archaea_p__Crenarchaeota_c__Thaumarchaeota_o__Nitrososphaerales_f__Nitrososphaeraceae_g__CandidatusNitrososphaera,k__Archaea_p__Euryarchaeota_c__Methanobacteria_o__Methanobacteriales_f__Methanobacteriaceae_g__Methanobrevibacter,k__Archaea_p__Euryarchaeota_c__Methanobacteria_o__Methanobacteriales_f__Methanobacteriaceae_g__Methanosphaera,k__Archaea_p__Euryarchaeota_c__Thermoplasmata_o__E2_f__[Methanomassiliicoccaceae]_g__vadinCA11,k__Bacteria_____,k__Bacteria_p__Acidobacteria_c__Acidobacteria-6_o__iii1-15_f___g__,k__Bacteria_p__Acidobacteria_c__Acidobacteria-6_o__iii1-15_f__mb2424_g__,...,k__Bacteria_p__Tenericutes_c__Mollicutes_o__Mycoplasmatales_f__Mycoplasmataceae_g__,k__Bacteria_p__Tenericutes_c__Mollicutes_o__RF39_f___g__,k__Bacteria_p__Tenericutes_c__RF3_o__ML615J-28_f___g__,k__Bacteria_p__Verrucomicrobia_c__Opitutae_o__HA64_f___g__,k__Bacteria_p__Verrucomicrobia_c__Opitutae_o__[Cerasicoccales]_f__[Cerasicoccaceae]_g__,k__Bacteria_p__Verrucomicrobia_c__Verruco-5_o__WCHB1-41_f__RFP12_g__,k__Bacteria_p__Verrucomicrobia_c__Verrucomicrobiae_o__Verrucomicrobiales_f__Verrucomicrobiaceae_g__Akkermansia,k__Bacteria_p__Verrucomicrobia_c__[Spartobacteria]_o__[Chthoniobacterales]_f__[Chthoniobacteraceae]_g__DA101,k__Bacteria_p__WPS-2_c___o___f___g__,k__Bacteria_p___c___o___f___g__
0,chimp_DRC_wild_campbell,chimp_DRC_wild_campbell,0.952381,NaN,0.400000,0.952381,0.222222,0.844795,1.000,NaN,...,1.000000,0.626793,1.000000,NaN,0.222222,0.684271,NaN,NaN,NaN,0.000000
1,chimp_DRC_wild_campbell,chimp_TZA_wild_moeller,NaN,NaN,0.211348,NaN,0.507755,0.944939,0.975,NaN,...,0.638889,0.794204,0.779603,NaN,0.507755,0.714956,NaN,NaN,NaN,0.012658
2,chimp_DRC_wild_campbell,chimp_USA_captive_campbell,1.000000,NaN,1.000000,1.000000,0.610979,0.966638,NaN,NaN,...,1.000000,0.866128,0.878175,NaN,0.610979,1.000000,NaN,NaN,NaN,1.000000
3,chimp_DRC_wild_campbell,chimp_USA_captive_moeller,NaN,NaN,1.000000,NaN,0.685714,0.985043,NaN,NaN,...,NaN,0.893772,0.950000,NaN,0.685714,1.000000,NaN,NaN,NaN,NaN
4,chimp_DRC_wild_campbell,douc_SGP_captive_clayton,NaN,NaN,1.000000,NaN,NaN,1.000000,NaN,NaN,...,NaN,0.991501,1.000000,NaN,NaN,1.000000,NaN,NaN,NaN,NaN


In [91]:
set(combined_pivot['Group1'])

{'chimp_DRC_wild_campbell',
 'chimp_TZA_wild_moeller',
 'chimp_USA_captive_campbell',
 'chimp_USA_captive_moeller',
 'douc_SGP_captive_clayton',
 'douc_USA_captive_clayton',
 'douc_VNM_semicaptive_clayton',
 'douc_VNM_wild_clayton',
 'gorilla_DRC_wild_campbell',
 'gorilla_USA_captive_campbell',
 'howler_CRI_captive_clayton',
 'howler_CRI_wild_clayton',
 'human_USA'}

In [92]:
set(combined_pivot['Group2'])

{'chimp_DRC_wild_campbell',
 'chimp_TZA_wild_moeller',
 'chimp_USA_captive_campbell',
 'chimp_USA_captive_moeller',
 'douc_SGP_captive_clayton',
 'douc_USA_captive_clayton',
 'douc_VNM_semicaptive_clayton',
 'douc_VNM_wild_clayton',
 'gorilla_DRC_wild_campbell',
 'gorilla_USA_captive_campbell',
 'howler_CRI_captive_clayton',
 'howler_CRI_wild_clayton',
 'human_USA'}

In [93]:
combined_pivot

Bacterial_Genus,Group1,Group2,Unassigned_____,k__Archaea_p__Crenarchaeota_c__Thaumarchaeota_o__Nitrososphaerales_f__Nitrososphaeraceae_g__CandidatusNitrososphaera,k__Archaea_p__Euryarchaeota_c__Methanobacteria_o__Methanobacteriales_f__Methanobacteriaceae_g__Methanobrevibacter,k__Archaea_p__Euryarchaeota_c__Methanobacteria_o__Methanobacteriales_f__Methanobacteriaceae_g__Methanosphaera,k__Archaea_p__Euryarchaeota_c__Thermoplasmata_o__E2_f__[Methanomassiliicoccaceae]_g__vadinCA11,k__Bacteria_____,k__Bacteria_p__Acidobacteria_c__Acidobacteria-6_o__iii1-15_f___g__,k__Bacteria_p__Acidobacteria_c__Acidobacteria-6_o__iii1-15_f__mb2424_g__,...,k__Bacteria_p__Tenericutes_c__Mollicutes_o__Mycoplasmatales_f__Mycoplasmataceae_g__,k__Bacteria_p__Tenericutes_c__Mollicutes_o__RF39_f___g__,k__Bacteria_p__Tenericutes_c__RF3_o__ML615J-28_f___g__,k__Bacteria_p__Verrucomicrobia_c__Opitutae_o__HA64_f___g__,k__Bacteria_p__Verrucomicrobia_c__Opitutae_o__[Cerasicoccales]_f__[Cerasicoccaceae]_g__,k__Bacteria_p__Verrucomicrobia_c__Verruco-5_o__WCHB1-41_f__RFP12_g__,k__Bacteria_p__Verrucomicrobia_c__Verrucomicrobiae_o__Verrucomicrobiales_f__Verrucomicrobiaceae_g__Akkermansia,k__Bacteria_p__Verrucomicrobia_c__[Spartobacteria]_o__[Chthoniobacterales]_f__[Chthoniobacteraceae]_g__DA101,k__Bacteria_p__WPS-2_c___o___f___g__,k__Bacteria_p___c___o___f___g__
0,chimp_DRC_wild_campbell,chimp_DRC_wild_campbell,0.952381,NaN,0.400000,0.952381,0.222222,0.844795,1.000,NaN,...,1.000000,0.626793,1.000000,NaN,0.222222,0.684271,NaN,NaN,NaN,0.000000
1,chimp_DRC_wild_campbell,chimp_TZA_wild_moeller,NaN,NaN,0.211348,NaN,0.507755,0.944939,0.975,NaN,...,0.638889,0.794204,0.779603,NaN,0.507755,0.714956,NaN,NaN,NaN,0.012658
2,chimp_DRC_wild_campbell,chimp_USA_captive_campbell,1.000000,NaN,1.000000,1.000000,0.610979,0.966638,NaN,NaN,...,1.000000,0.866128,0.878175,NaN,0.610979,1.000000,NaN,NaN,NaN,1.000000
3,chimp_DRC_wild_campbell,chimp_USA_captive_moeller,NaN,NaN,1.000000,NaN,0.685714,0.985043,NaN,NaN,...,NaN,0.893772,0.950000,NaN,0.685714,1.000000,NaN,NaN,NaN,NaN
4,chimp_DRC_wild_campbell,douc_SGP_captive_clayton,NaN,NaN,1.000000,NaN,NaN,1.000000,NaN,NaN,...,NaN,0.991501,1.000000,NaN,NaN,1.000000,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164,human_USA,gorilla_DRC_wild_campbell,NaN,NaN,NaN,NaN,1.000000,0.999695,NaN,NaN,...,NaN,0.980359,1.000000,NaN,1.000000,NaN,NaN,NaN,NaN,1.000000
165,human_USA,gorilla_USA_captive_campbell,NaN,NaN,NaN,NaN,1.000000,0.994426,NaN,NaN,...,NaN,0.979166,0.984683,NaN,1.000000,NaN,0.532164,NaN,NaN,NaN
166,human_USA,howler_CRI_captive_clayton,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,...,NaN,0.999442,NaN,NaN,NaN,NaN,0.532164,NaN,NaN,NaN
167,human_USA,howler_CRI_wild_clayton,NaN,NaN,NaN,NaN,1.000000,1.000000,NaN,NaN,...,NaN,0.994831,NaN,NaN,1.000000,NaN,0.970760,0.666667,NaN,NaN


## Load target group comparisons

This file should list just the specific group comparisons we want to look at.

In [94]:
target_grp_df = pd.read_csv('./data/host_specificity_score_populations_american_gut.txt', sep='\t')

In [95]:
target_grp_df

,captive_population,human_population,wild_population
0,douc_USA_captive_clayton,human_USA,douc_VNM_wild_clayton
1,douc_SGP_captive_clayton,human_USA,douc_VNM_wild_clayton
2,chimp_USA_captive_moeller,human_USA,chimp_TZA_wild_moeller
3,chimp_USA_captive_campbell,human_USA,chimp_DRC_wild_campbell
4,howler_CRI_captive_clayton,human_USA,howler_CRI_wild_clayton
5,gorilla_USA_captive_campbell,human_USA,gorilla_DRC_wild_campbell
6,douc_VNM_semicaptive_clayton,human_USA,douc_VNM_wild_clayton


## Filter data list to target comparisons

Update the aggregated pivot table to include the comparison number and comparison direction (captive to human or captive to wild) as additional columns

In [96]:
combined_pivot['comparison_num'] = 'None'
combined_pivot['comparison_dir'] = 'None'

for i, row in target_grp_df.iterrows():
    combined_pivot.loc[(combined_pivot['Group1'] == row['captive_population']) &
                       (combined_pivot['Group2'] == row['human_population']),
                       'comparison_num'] = i
    combined_pivot.loc[(combined_pivot['Group1'] == row['captive_population']) &
                       (combined_pivot['Group2'] == row['human_population']),
                       'comparison_dir'] = 'human'
    combined_pivot.loc[(combined_pivot['Group1'] == row['captive_population']) &
                       (combined_pivot['Group2'] == row['wild_population']),
                       'comparison_num'] = i
    combined_pivot.loc[(combined_pivot['Group1'] == row['captive_population']) &
                       (combined_pivot['Group2'] == row['wild_population']),
                       'comparison_dir'] = 'wild'

Remove non-target comparisons from the table

In [97]:
combined_pivot_filtered = combined_pivot.loc[(combined_pivot['comparison_num'] != 'None'),]

In [98]:
combined_pivot_filtered

Bacterial_Genus,Group1,Group2,Unassigned_____,k__Archaea_p__Crenarchaeota_c__Thaumarchaeota_o__Nitrososphaerales_f__Nitrososphaeraceae_g__CandidatusNitrososphaera,k__Archaea_p__Euryarchaeota_c__Methanobacteria_o__Methanobacteriales_f__Methanobacteriaceae_g__Methanobrevibacter,k__Archaea_p__Euryarchaeota_c__Methanobacteria_o__Methanobacteriales_f__Methanobacteriaceae_g__Methanosphaera,k__Archaea_p__Euryarchaeota_c__Thermoplasmata_o__E2_f__[Methanomassiliicoccaceae]_g__vadinCA11,k__Bacteria_____,k__Bacteria_p__Acidobacteria_c__Acidobacteria-6_o__iii1-15_f___g__,k__Bacteria_p__Acidobacteria_c__Acidobacteria-6_o__iii1-15_f__mb2424_g__,...,k__Bacteria_p__Tenericutes_c__RF3_o__ML615J-28_f___g__,k__Bacteria_p__Verrucomicrobia_c__Opitutae_o__HA64_f___g__,k__Bacteria_p__Verrucomicrobia_c__Opitutae_o__[Cerasicoccales]_f__[Cerasicoccaceae]_g__,k__Bacteria_p__Verrucomicrobia_c__Verruco-5_o__WCHB1-41_f__RFP12_g__,k__Bacteria_p__Verrucomicrobia_c__Verrucomicrobiae_o__Verrucomicrobiales_f__Verrucomicrobiaceae_g__Akkermansia,k__Bacteria_p__Verrucomicrobia_c__[Spartobacteria]_o__[Chthoniobacterales]_f__[Chthoniobacteraceae]_g__DA101,k__Bacteria_p__WPS-2_c___o___f___g__,k__Bacteria_p___c___o___f___g__,comparison_num,comparison_dir
26,chimp_USA_captive_campbell,chimp_DRC_wild_campbell,1.0,NaN,1.0,1.0,0.610979,0.966638,NaN,NaN,...,0.878175,NaN,0.610979,1.000000,NaN,NaN,NaN,1.0,3,wild
38,chimp_USA_captive_campbell,human_USA,NaN,NaN,NaN,NaN,1.000000,0.998805,NaN,NaN,...,0.980938,NaN,1.000000,NaN,0.298246,NaN,NaN,1.0,3,human
40,chimp_USA_captive_moeller,chimp_TZA_wild_moeller,NaN,NaN,1.0,NaN,0.513061,0.940518,NaN,NaN,...,0.921071,NaN,0.513061,0.926907,0.666667,NaN,0.210526,NaN,2,wild
51,chimp_USA_captive_moeller,human_USA,NaN,NaN,NaN,NaN,1.000000,0.998232,NaN,NaN,...,0.965000,NaN,1.000000,NaN,0.376218,NaN,NaN,NaN,2,human
59,douc_SGP_captive_clayton,douc_VNM_wild_clayton,NaN,NaN,NaN,NaN,NaN,0.966596,NaN,NaN,...,0.690848,NaN,NaN,NaN,0.862233,NaN,NaN,NaN,1,wild
64,douc_SGP_captive_clayton,human_USA,NaN,NaN,NaN,NaN,NaN,0.998618,NaN,NaN,...,1.000000,1.0,NaN,NaN,0.401949,NaN,NaN,NaN,1,human
72,douc_USA_captive_clayton,douc_VNM_wild_clayton,NaN,NaN,NaN,NaN,1.000000,1.000000,NaN,NaN,...,1.000000,NaN,1.000000,NaN,0.897547,NaN,NaN,NaN,0,wild
77,douc_USA_captive_clayton,human_USA,NaN,NaN,NaN,NaN,1.000000,0.986176,NaN,NaN,...,1.000000,NaN,1.000000,NaN,0.364411,NaN,NaN,NaN,0,human
85,douc_VNM_semicaptive_clayton,douc_VNM_wild_clayton,NaN,NaN,NaN,NaN,1.000000,0.905775,NaN,NaN,...,0.730686,NaN,1.000000,NaN,0.321829,NaN,NaN,NaN,6,wild
90,douc_VNM_semicaptive_clayton,human_USA,NaN,NaN,NaN,NaN,1.000000,1.000000,NaN,NaN,...,1.000000,NaN,1.000000,NaN,0.961014,NaN,NaN,NaN,6,human


## Calculate HSS

First, transpose and set column indexed on the combined filtered dataframe so that taxon is the row index and comparison number / comparison direction are a column MultiIndex

In [99]:
combined_pivot_filtered_t = combined_pivot_filtered.drop(columns=['Group1', 'Group2']).set_index(['comparison_num',
                                                                            'comparison_dir']).transpose()

In [100]:
combined_pivot_filtered_t.head()

comparison_num                                             3               2  \
comparison_dir                                          wild human      wild   
Bacterial_Genus                                                                
Unassigned_____                                     1.000000   NaN       NaN   
k__Archaea_p__Crenarchaeota_c__Thaumarchaeota_o...       NaN   NaN       NaN   
k__Archaea_p__Euryarchaeota_c__Methanobacteria_...  1.000000   NaN  1.000000   
k__Archaea_p__Euryarchaeota_c__Methanobacteria_...  1.000000   NaN       NaN   
k__Archaea_p__Euryarchaeota_c__Thermoplasmata_o...  0.610979   1.0  0.513061   

comparison_num                                              1          0  \
comparison_dir                                     human wild human wild   
Bacterial_Genus                                                            
Unassigned_____                                      NaN  NaN   NaN  NaN   
k__Archaea_p__Crenarchaeota_c__Thaumarchaeota_o...   NaN  NaN   NaN  NaN   
k__Archaea_p__Euryarchaeota_c__Methanobacteria_...   NaN  NaN   NaN  NaN   
k__Archaea_p__Euryarchaeota_c__Methanobacteria_...   NaN  NaN   NaN  NaN   
k__Archaea_p__Euryarchaeota_c__Thermoplasmata_o...   1.0  NaN   NaN  1.0   

comparison_num                                              6               5  \
comparison_dir                                     human wild human      wild   
Bacterial_Genus                                                                 
Unassigned_____                                      NaN  NaN   NaN       NaN   
k__Archaea_p__Crenarchaeota_c__Thaumarchaeota_o...   NaN  NaN   NaN       NaN   
k__Archaea_p__Euryarchaeota_c__Methanobacteria_...   NaN  NaN   NaN       NaN   
k__Archaea_p__Euryarchaeota_c__Methanobacteria_...   NaN  NaN   NaN       NaN   
k__Archaea_p__Euryarchaeota_c__Thermoplasmata_o...   1.0  1.0   1.0  0.840678   

comparison_num                                              4        
comparison_dir                                     human wild human  
Bacterial_Genus                                                      
Unassigned_____                                      NaN  NaN   NaN  
k__Archaea_p__Crenarchaeota_c__Thaumarchaeota_o...   NaN  NaN   NaN  
k__Archaea_p__Euryarchaeota_c__Methanobacteria_...   NaN  NaN   NaN  
k__Archaea_p__Euryarchaeota_c__Methanobacteria_...   NaN  NaN   NaN  
k__Archaea_p__Euryarchaeota_c__Thermoplasmata_o...   1.0  NaN   NaN

In [101]:
target_grps = target_grp_df.copy()

In [102]:
target_grps.drop(columns=['human_population', 'wild_population'], 
                inplace=True)

In [103]:
HSS_df = pd.DataFrame(index=combined_pivot_filtered_t.index, 
                      columns=target_grps.index)

for i, row in target_grps.iterrows():
    HSS = combined_pivot_filtered_t.loc[:,(i,
                     'human')]/combined_pivot_filtered_t.loc[:,(i,
                                          'wild')]
    HSS_df[i] = HSS

In [104]:
HSS_df.head()

,0,1,2,3,4,5,6
Bacterial_Genus,,,,,,,
Unassigned_____,NaN,NaN,NaN,NaN,NaN,NaN,NaN
k__Archaea_p__Crenarchaeota_c__Thaumarchaeota_o__Nitrososphaerales_f__Nitrososphaeraceae_g__CandidatusNitrososphaera,NaN,NaN,NaN,NaN,NaN,NaN,NaN
k__Archaea_p__Euryarchaeota_c__Methanobacteria_o__Methanobacteriales_f__Methanobacteriaceae_g__Methanobrevibacter,NaN,NaN,NaN,NaN,NaN,NaN,NaN
k__Archaea_p__Euryarchaeota_c__Methanobacteria_o__Methanobacteriales_f__Methanobacteriaceae_g__Methanosphaera,NaN,NaN,NaN,NaN,NaN,NaN,NaN
k__Archaea_p__Euryarchaeota_c__Thermoplasmata_o__E2_f__[Methanomassiliicoccaceae]_g__vadinCA11,1.0,NaN,1.949084,1.636718,NaN,1.189516,1.0


Make this pretty; add back in the comparison names

In [105]:
HSS_df_t = HSS_df.transpose()
HSS_df_t

Bacterial_Genus,Unassigned_____,k__Archaea_p__Crenarchaeota_c__Thaumarchaeota_o__Nitrososphaerales_f__Nitrososphaeraceae_g__CandidatusNitrososphaera,k__Archaea_p__Euryarchaeota_c__Methanobacteria_o__Methanobacteriales_f__Methanobacteriaceae_g__Methanobrevibacter,k__Archaea_p__Euryarchaeota_c__Methanobacteria_o__Methanobacteriales_f__Methanobacteriaceae_g__Methanosphaera,k__Archaea_p__Euryarchaeota_c__Thermoplasmata_o__E2_f__[Methanomassiliicoccaceae]_g__vadinCA11,k__Bacteria_____,k__Bacteria_p__Acidobacteria_c__Acidobacteria-6_o__iii1-15_f___g__,k__Bacteria_p__Acidobacteria_c__Acidobacteria-6_o__iii1-15_f__mb2424_g__,k__Bacteria_p__Actinobacteria____,k__Bacteria_p__Actinobacteria_c__Acidimicrobiia_o__Acidimicrobiales_f___g__,...,k__Bacteria_p__Tenericutes_c__Mollicutes_o__Mycoplasmatales_f__Mycoplasmataceae_g__,k__Bacteria_p__Tenericutes_c__Mollicutes_o__RF39_f___g__,k__Bacteria_p__Tenericutes_c__RF3_o__ML615J-28_f___g__,k__Bacteria_p__Verrucomicrobia_c__Opitutae_o__HA64_f___g__,k__Bacteria_p__Verrucomicrobia_c__Opitutae_o__[Cerasicoccales]_f__[Cerasicoccaceae]_g__,k__Bacteria_p__Verrucomicrobia_c__Verruco-5_o__WCHB1-41_f__RFP12_g__,k__Bacteria_p__Verrucomicrobia_c__Verrucomicrobiae_o__Verrucomicrobiales_f__Verrucomicrobiaceae_g__Akkermansia,k__Bacteria_p__Verrucomicrobia_c__[Spartobacteria]_o__[Chthoniobacterales]_f__[Chthoniobacteraceae]_g__DA101,k__Bacteria_p__WPS-2_c___o___f___g__,k__Bacteria_p___c___o___f___g__
0,NaN,NaN,NaN,NaN,1.000000,0.986176,NaN,NaN,NaN,NaN,...,NaN,1.001087,1.000000,NaN,1.000000,NaN,0.406008,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,1.033128,NaN,NaN,NaN,NaN,...,NaN,1.034226,1.447496,NaN,NaN,NaN,0.466172,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,1.949084,1.061364,NaN,NaN,NaN,NaN,...,NaN,1.098157,1.047693,NaN,1.949084,NaN,0.564327,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,1.636718,1.033277,NaN,NaN,NaN,NaN,...,NaN,1.126103,1.117020,NaN,1.636718,NaN,NaN,NaN,NaN,1.0
4,NaN,NaN,NaN,NaN,NaN,1.197175,NaN,NaN,NaN,NaN,...,NaN,1.072728,NaN,NaN,NaN,NaN,0.746411,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,1.189516,1.002035,NaN,NaN,NaN,NaN,...,NaN,1.152631,0.984683,NaN,1.189516,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,1.000000,1.104027,NaN,NaN,NaN,NaN,...,NaN,1.250603,1.368577,NaN,1.000000,NaN,2.986096,NaN,NaN,NaN


In [106]:
target_grp_df.columns

Index(['captive_population', 'human_population', 'wild_population'], dtype='object')

In [107]:
final_HSS_table = HSS_df_t.merge(target_grp_df,
                      left_index=True,
                      right_index=True).set_index(list(target_grp_df.columns)).transpose()

In [108]:
final_HSS_table.head()

captive_population,douc_USA_captive_clayton,douc_SGP_captive_clayton,chimp_USA_captive_moeller,chimp_USA_captive_campbell,howler_CRI_captive_clayton,gorilla_USA_captive_campbell,douc_VNM_semicaptive_clayton
human_population,human_USA,human_USA,human_USA,human_USA,human_USA,human_USA,human_USA
wild_population,douc_VNM_wild_clayton,douc_VNM_wild_clayton,chimp_TZA_wild_moeller,chimp_DRC_wild_campbell,howler_CRI_wild_clayton,gorilla_DRC_wild_campbell,douc_VNM_wild_clayton
Unassigned_____,NaN,NaN,NaN,NaN,NaN,NaN,NaN
k__Archaea_p__Crenarchaeota_c__Thaumarchaeota_o__Nitrososphaerales_f__Nitrososphaeraceae_g__CandidatusNitrososphaera,NaN,NaN,NaN,NaN,NaN,NaN,NaN
k__Archaea_p__Euryarchaeota_c__Methanobacteria_o__Methanobacteriales_f__Methanobacteriaceae_g__Methanobrevibacter,NaN,NaN,NaN,NaN,NaN,NaN,NaN
k__Archaea_p__Euryarchaeota_c__Methanobacteria_o__Methanobacteriales_f__Methanobacteriaceae_g__Methanosphaera,NaN,NaN,NaN,NaN,NaN,NaN,NaN
k__Archaea_p__Euryarchaeota_c__Thermoplasmata_o__E2_f__[Methanomassiliicoccaceae]_g__vadinCA11,1.0,NaN,1.949084,1.636718,NaN,1.189516,1.0


Write to file

In [109]:
outdir = join('output', 'results')
makedirs(outdir, exist_ok=True)
final_HSS_table.to_csv(join(outdir, 'hss-amgut.txt'), sep='\t')

## Calculate Microbiota Convergence Score

The MCS is the ratio of [captive vs human] / [wild vs human]

## Filter data list to target comparisons

Update the aggregated pivot table to include the comparison number and comparison direction (captive to human or human to wild) as additional columns

In [110]:
combined_pivot['comparison_num'] = 'None'
combined_pivot['comparison_dir'] = 'None'

In [111]:
combined_pivot.head()

Bacterial_Genus,Group1,Group2,Unassigned_____,k__Archaea_p__Crenarchaeota_c__Thaumarchaeota_o__Nitrososphaerales_f__Nitrososphaeraceae_g__CandidatusNitrososphaera,k__Archaea_p__Euryarchaeota_c__Methanobacteria_o__Methanobacteriales_f__Methanobacteriaceae_g__Methanobrevibacter,k__Archaea_p__Euryarchaeota_c__Methanobacteria_o__Methanobacteriales_f__Methanobacteriaceae_g__Methanosphaera,k__Archaea_p__Euryarchaeota_c__Thermoplasmata_o__E2_f__[Methanomassiliicoccaceae]_g__vadinCA11,k__Bacteria_____,k__Bacteria_p__Acidobacteria_c__Acidobacteria-6_o__iii1-15_f___g__,k__Bacteria_p__Acidobacteria_c__Acidobacteria-6_o__iii1-15_f__mb2424_g__,...,k__Bacteria_p__Tenericutes_c__RF3_o__ML615J-28_f___g__,k__Bacteria_p__Verrucomicrobia_c__Opitutae_o__HA64_f___g__,k__Bacteria_p__Verrucomicrobia_c__Opitutae_o__[Cerasicoccales]_f__[Cerasicoccaceae]_g__,k__Bacteria_p__Verrucomicrobia_c__Verruco-5_o__WCHB1-41_f__RFP12_g__,k__Bacteria_p__Verrucomicrobia_c__Verrucomicrobiae_o__Verrucomicrobiales_f__Verrucomicrobiaceae_g__Akkermansia,k__Bacteria_p__Verrucomicrobia_c__[Spartobacteria]_o__[Chthoniobacterales]_f__[Chthoniobacteraceae]_g__DA101,k__Bacteria_p__WPS-2_c___o___f___g__,k__Bacteria_p___c___o___f___g__,comparison_num,comparison_dir
0,chimp_DRC_wild_campbell,chimp_DRC_wild_campbell,0.952381,NaN,0.400000,0.952381,0.222222,0.844795,1.000,NaN,...,1.000000,NaN,0.222222,0.684271,NaN,NaN,NaN,0.000000,None,None
1,chimp_DRC_wild_campbell,chimp_TZA_wild_moeller,NaN,NaN,0.211348,NaN,0.507755,0.944939,0.975,NaN,...,0.779603,NaN,0.507755,0.714956,NaN,NaN,NaN,0.012658,None,None
2,chimp_DRC_wild_campbell,chimp_USA_captive_campbell,1.000000,NaN,1.000000,1.000000,0.610979,0.966638,NaN,NaN,...,0.878175,NaN,0.610979,1.000000,NaN,NaN,NaN,1.000000,None,None
3,chimp_DRC_wild_campbell,chimp_USA_captive_moeller,NaN,NaN,1.000000,NaN,0.685714,0.985043,NaN,NaN,...,0.950000,NaN,0.685714,1.000000,NaN,NaN,NaN,NaN,None,None
4,chimp_DRC_wild_campbell,douc_SGP_captive_clayton,NaN,NaN,1.000000,NaN,NaN,1.000000,NaN,NaN,...,1.000000,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,None,None


In [112]:
target_grp_df

,captive_population,human_population,wild_population
0,douc_USA_captive_clayton,human_USA,douc_VNM_wild_clayton
1,douc_SGP_captive_clayton,human_USA,douc_VNM_wild_clayton
2,chimp_USA_captive_moeller,human_USA,chimp_TZA_wild_moeller
3,chimp_USA_captive_campbell,human_USA,chimp_DRC_wild_campbell
4,howler_CRI_captive_clayton,human_USA,howler_CRI_wild_clayton
5,gorilla_USA_captive_campbell,human_USA,gorilla_DRC_wild_campbell
6,douc_VNM_semicaptive_clayton,human_USA,douc_VNM_wild_clayton


In [113]:
for i, row in target_grp_df.iterrows():
    combined_pivot.loc[(combined_pivot['Group1'] == row['captive_population']) &
                       (combined_pivot['Group2'] == row['human_population']),
                       'comparison_num'] = i
    combined_pivot.loc[(combined_pivot['Group1'] == row['captive_population']) &
                       (combined_pivot['Group2'] == row['human_population']),
                       'comparison_dir'] = 'captive-human'
    combined_pivot.loc[(combined_pivot['Group1'] == row['wild_population']) &
                       (combined_pivot['Group2'] == row['human_population']),
                       'comparison_num'] = i
    combined_pivot.loc[(combined_pivot['Group1'] == row['wild_population']) &
                       (combined_pivot['Group2'] == row['human_population']),
                       'comparison_dir'] = 'wild-human'

In [114]:
combined_pivot.loc[(combined_pivot['comparison_num'] != 'None'),]

Bacterial_Genus,Group1,Group2,Unassigned_____,k__Archaea_p__Crenarchaeota_c__Thaumarchaeota_o__Nitrososphaerales_f__Nitrososphaeraceae_g__CandidatusNitrososphaera,k__Archaea_p__Euryarchaeota_c__Methanobacteria_o__Methanobacteriales_f__Methanobacteriaceae_g__Methanobrevibacter,k__Archaea_p__Euryarchaeota_c__Methanobacteria_o__Methanobacteriales_f__Methanobacteriaceae_g__Methanosphaera,k__Archaea_p__Euryarchaeota_c__Thermoplasmata_o__E2_f__[Methanomassiliicoccaceae]_g__vadinCA11,k__Bacteria_____,k__Bacteria_p__Acidobacteria_c__Acidobacteria-6_o__iii1-15_f___g__,k__Bacteria_p__Acidobacteria_c__Acidobacteria-6_o__iii1-15_f__mb2424_g__,...,k__Bacteria_p__Tenericutes_c__RF3_o__ML615J-28_f___g__,k__Bacteria_p__Verrucomicrobia_c__Opitutae_o__HA64_f___g__,k__Bacteria_p__Verrucomicrobia_c__Opitutae_o__[Cerasicoccales]_f__[Cerasicoccaceae]_g__,k__Bacteria_p__Verrucomicrobia_c__Verruco-5_o__WCHB1-41_f__RFP12_g__,k__Bacteria_p__Verrucomicrobia_c__Verrucomicrobiae_o__Verrucomicrobiales_f__Verrucomicrobiaceae_g__Akkermansia,k__Bacteria_p__Verrucomicrobia_c__[Spartobacteria]_o__[Chthoniobacterales]_f__[Chthoniobacteraceae]_g__DA101,k__Bacteria_p__WPS-2_c___o___f___g__,k__Bacteria_p___c___o___f___g__,comparison_num,comparison_dir
12,chimp_DRC_wild_campbell,human_USA,NaN,NaN,NaN,NaN,1.0,1.000000,NaN,NaN,...,1.000000,NaN,1.0,NaN,NaN,NaN,NaN,1.0,3,wild-human
25,chimp_TZA_wild_moeller,human_USA,NaN,NaN,NaN,NaN,1.0,0.996629,NaN,1.0,...,0.979537,NaN,1.0,NaN,0.589912,1.000000,NaN,1.0,2,wild-human
38,chimp_USA_captive_campbell,human_USA,NaN,NaN,NaN,NaN,1.0,0.998805,NaN,NaN,...,0.980938,NaN,1.0,NaN,0.298246,NaN,NaN,1.0,3,captive-human
51,chimp_USA_captive_moeller,human_USA,NaN,NaN,NaN,NaN,1.0,0.998232,NaN,NaN,...,0.965000,NaN,1.0,NaN,0.376218,NaN,NaN,NaN,2,captive-human
64,douc_SGP_captive_clayton,human_USA,NaN,NaN,NaN,NaN,NaN,0.998618,NaN,NaN,...,1.000000,1.0,NaN,NaN,0.401949,NaN,NaN,NaN,1,captive-human
77,douc_USA_captive_clayton,human_USA,NaN,NaN,NaN,NaN,1.0,0.986176,NaN,NaN,...,1.000000,NaN,1.0,NaN,0.364411,NaN,NaN,NaN,0,captive-human
90,douc_VNM_semicaptive_clayton,human_USA,NaN,NaN,NaN,NaN,1.0,1.000000,NaN,NaN,...,1.000000,NaN,1.0,NaN,0.961014,NaN,NaN,NaN,6,captive-human
103,douc_VNM_wild_clayton,human_USA,NaN,NaN,NaN,NaN,1.0,1.000000,NaN,1.0,...,0.992835,NaN,1.0,NaN,1.000000,NaN,NaN,NaN,6,wild-human
116,gorilla_DRC_wild_campbell,human_USA,NaN,NaN,NaN,NaN,1.0,0.999695,NaN,NaN,...,1.000000,NaN,1.0,NaN,NaN,NaN,NaN,1.0,5,wild-human
129,gorilla_USA_captive_campbell,human_USA,NaN,NaN,NaN,NaN,1.0,0.994426,NaN,NaN,...,0.984683,NaN,1.0,NaN,0.532164,NaN,NaN,NaN,5,captive-human


Remove non-target comparisons from the table

In [115]:
combined_pivot_filtered_MCS = combined_pivot.loc[(combined_pivot['comparison_num'] != 'None'),]

In [116]:
combined_pivot_filtered_MCS

Bacterial_Genus,Group1,Group2,Unassigned_____,k__Archaea_p__Crenarchaeota_c__Thaumarchaeota_o__Nitrososphaerales_f__Nitrososphaeraceae_g__CandidatusNitrososphaera,k__Archaea_p__Euryarchaeota_c__Methanobacteria_o__Methanobacteriales_f__Methanobacteriaceae_g__Methanobrevibacter,k__Archaea_p__Euryarchaeota_c__Methanobacteria_o__Methanobacteriales_f__Methanobacteriaceae_g__Methanosphaera,k__Archaea_p__Euryarchaeota_c__Thermoplasmata_o__E2_f__[Methanomassiliicoccaceae]_g__vadinCA11,k__Bacteria_____,k__Bacteria_p__Acidobacteria_c__Acidobacteria-6_o__iii1-15_f___g__,k__Bacteria_p__Acidobacteria_c__Acidobacteria-6_o__iii1-15_f__mb2424_g__,...,k__Bacteria_p__Tenericutes_c__RF3_o__ML615J-28_f___g__,k__Bacteria_p__Verrucomicrobia_c__Opitutae_o__HA64_f___g__,k__Bacteria_p__Verrucomicrobia_c__Opitutae_o__[Cerasicoccales]_f__[Cerasicoccaceae]_g__,k__Bacteria_p__Verrucomicrobia_c__Verruco-5_o__WCHB1-41_f__RFP12_g__,k__Bacteria_p__Verrucomicrobia_c__Verrucomicrobiae_o__Verrucomicrobiales_f__Verrucomicrobiaceae_g__Akkermansia,k__Bacteria_p__Verrucomicrobia_c__[Spartobacteria]_o__[Chthoniobacterales]_f__[Chthoniobacteraceae]_g__DA101,k__Bacteria_p__WPS-2_c___o___f___g__,k__Bacteria_p___c___o___f___g__,comparison_num,comparison_dir
12,chimp_DRC_wild_campbell,human_USA,NaN,NaN,NaN,NaN,1.0,1.000000,NaN,NaN,...,1.000000,NaN,1.0,NaN,NaN,NaN,NaN,1.0,3,wild-human
25,chimp_TZA_wild_moeller,human_USA,NaN,NaN,NaN,NaN,1.0,0.996629,NaN,1.0,...,0.979537,NaN,1.0,NaN,0.589912,1.000000,NaN,1.0,2,wild-human
38,chimp_USA_captive_campbell,human_USA,NaN,NaN,NaN,NaN,1.0,0.998805,NaN,NaN,...,0.980938,NaN,1.0,NaN,0.298246,NaN,NaN,1.0,3,captive-human
51,chimp_USA_captive_moeller,human_USA,NaN,NaN,NaN,NaN,1.0,0.998232,NaN,NaN,...,0.965000,NaN,1.0,NaN,0.376218,NaN,NaN,NaN,2,captive-human
64,douc_SGP_captive_clayton,human_USA,NaN,NaN,NaN,NaN,NaN,0.998618,NaN,NaN,...,1.000000,1.0,NaN,NaN,0.401949,NaN,NaN,NaN,1,captive-human
77,douc_USA_captive_clayton,human_USA,NaN,NaN,NaN,NaN,1.0,0.986176,NaN,NaN,...,1.000000,NaN,1.0,NaN,0.364411,NaN,NaN,NaN,0,captive-human
90,douc_VNM_semicaptive_clayton,human_USA,NaN,NaN,NaN,NaN,1.0,1.000000,NaN,NaN,...,1.000000,NaN,1.0,NaN,0.961014,NaN,NaN,NaN,6,captive-human
103,douc_VNM_wild_clayton,human_USA,NaN,NaN,NaN,NaN,1.0,1.000000,NaN,1.0,...,0.992835,NaN,1.0,NaN,1.000000,NaN,NaN,NaN,6,wild-human
116,gorilla_DRC_wild_campbell,human_USA,NaN,NaN,NaN,NaN,1.0,0.999695,NaN,NaN,...,1.000000,NaN,1.0,NaN,NaN,NaN,NaN,1.0,5,wild-human
129,gorilla_USA_captive_campbell,human_USA,NaN,NaN,NaN,NaN,1.0,0.994426,NaN,NaN,...,0.984683,NaN,1.0,NaN,0.532164,NaN,NaN,NaN,5,captive-human


## Calculate MCS

First, transpose and set column indexed on the combined filtered dataframe so that taxon is the row index and comparison number / comparison direction are a column MultiIndex

In [117]:
combined_pivot_filtered_MCS

Bacterial_Genus,Group1,Group2,Unassigned_____,k__Archaea_p__Crenarchaeota_c__Thaumarchaeota_o__Nitrososphaerales_f__Nitrososphaeraceae_g__CandidatusNitrososphaera,k__Archaea_p__Euryarchaeota_c__Methanobacteria_o__Methanobacteriales_f__Methanobacteriaceae_g__Methanobrevibacter,k__Archaea_p__Euryarchaeota_c__Methanobacteria_o__Methanobacteriales_f__Methanobacteriaceae_g__Methanosphaera,k__Archaea_p__Euryarchaeota_c__Thermoplasmata_o__E2_f__[Methanomassiliicoccaceae]_g__vadinCA11,k__Bacteria_____,k__Bacteria_p__Acidobacteria_c__Acidobacteria-6_o__iii1-15_f___g__,k__Bacteria_p__Acidobacteria_c__Acidobacteria-6_o__iii1-15_f__mb2424_g__,...,k__Bacteria_p__Tenericutes_c__RF3_o__ML615J-28_f___g__,k__Bacteria_p__Verrucomicrobia_c__Opitutae_o__HA64_f___g__,k__Bacteria_p__Verrucomicrobia_c__Opitutae_o__[Cerasicoccales]_f__[Cerasicoccaceae]_g__,k__Bacteria_p__Verrucomicrobia_c__Verruco-5_o__WCHB1-41_f__RFP12_g__,k__Bacteria_p__Verrucomicrobia_c__Verrucomicrobiae_o__Verrucomicrobiales_f__Verrucomicrobiaceae_g__Akkermansia,k__Bacteria_p__Verrucomicrobia_c__[Spartobacteria]_o__[Chthoniobacterales]_f__[Chthoniobacteraceae]_g__DA101,k__Bacteria_p__WPS-2_c___o___f___g__,k__Bacteria_p___c___o___f___g__,comparison_num,comparison_dir
12,chimp_DRC_wild_campbell,human_USA,NaN,NaN,NaN,NaN,1.0,1.000000,NaN,NaN,...,1.000000,NaN,1.0,NaN,NaN,NaN,NaN,1.0,3,wild-human
25,chimp_TZA_wild_moeller,human_USA,NaN,NaN,NaN,NaN,1.0,0.996629,NaN,1.0,...,0.979537,NaN,1.0,NaN,0.589912,1.000000,NaN,1.0,2,wild-human
38,chimp_USA_captive_campbell,human_USA,NaN,NaN,NaN,NaN,1.0,0.998805,NaN,NaN,...,0.980938,NaN,1.0,NaN,0.298246,NaN,NaN,1.0,3,captive-human
51,chimp_USA_captive_moeller,human_USA,NaN,NaN,NaN,NaN,1.0,0.998232,NaN,NaN,...,0.965000,NaN,1.0,NaN,0.376218,NaN,NaN,NaN,2,captive-human
64,douc_SGP_captive_clayton,human_USA,NaN,NaN,NaN,NaN,NaN,0.998618,NaN,NaN,...,1.000000,1.0,NaN,NaN,0.401949,NaN,NaN,NaN,1,captive-human
77,douc_USA_captive_clayton,human_USA,NaN,NaN,NaN,NaN,1.0,0.986176,NaN,NaN,...,1.000000,NaN,1.0,NaN,0.364411,NaN,NaN,NaN,0,captive-human
90,douc_VNM_semicaptive_clayton,human_USA,NaN,NaN,NaN,NaN,1.0,1.000000,NaN,NaN,...,1.000000,NaN,1.0,NaN,0.961014,NaN,NaN,NaN,6,captive-human
103,douc_VNM_wild_clayton,human_USA,NaN,NaN,NaN,NaN,1.0,1.000000,NaN,1.0,...,0.992835,NaN,1.0,NaN,1.000000,NaN,NaN,NaN,6,wild-human
116,gorilla_DRC_wild_campbell,human_USA,NaN,NaN,NaN,NaN,1.0,0.999695,NaN,NaN,...,1.000000,NaN,1.0,NaN,NaN,NaN,NaN,1.0,5,wild-human
129,gorilla_USA_captive_campbell,human_USA,NaN,NaN,NaN,NaN,1.0,0.994426,NaN,NaN,...,0.984683,NaN,1.0,NaN,0.532164,NaN,NaN,NaN,5,captive-human


Because we are re-using the Douc-Wild comparisons, we need to manually replicate those rows in the data frame.

In [118]:
douc_0_wild = combined_pivot_filtered_MCS.loc[
        (combined_pivot_filtered_MCS['comparison_num'] == 6) &
        (combined_pivot_filtered_MCS['comparison_dir'] == 'wild-human')].reset_index(drop=True)

In [119]:
douc_0_wild

Bacterial_Genus,Group1,Group2,Unassigned_____,k__Archaea_p__Crenarchaeota_c__Thaumarchaeota_o__Nitrososphaerales_f__Nitrososphaeraceae_g__CandidatusNitrososphaera,k__Archaea_p__Euryarchaeota_c__Methanobacteria_o__Methanobacteriales_f__Methanobacteriaceae_g__Methanobrevibacter,k__Archaea_p__Euryarchaeota_c__Methanobacteria_o__Methanobacteriales_f__Methanobacteriaceae_g__Methanosphaera,k__Archaea_p__Euryarchaeota_c__Thermoplasmata_o__E2_f__[Methanomassiliicoccaceae]_g__vadinCA11,k__Bacteria_____,k__Bacteria_p__Acidobacteria_c__Acidobacteria-6_o__iii1-15_f___g__,k__Bacteria_p__Acidobacteria_c__Acidobacteria-6_o__iii1-15_f__mb2424_g__,...,k__Bacteria_p__Tenericutes_c__RF3_o__ML615J-28_f___g__,k__Bacteria_p__Verrucomicrobia_c__Opitutae_o__HA64_f___g__,k__Bacteria_p__Verrucomicrobia_c__Opitutae_o__[Cerasicoccales]_f__[Cerasicoccaceae]_g__,k__Bacteria_p__Verrucomicrobia_c__Verruco-5_o__WCHB1-41_f__RFP12_g__,k__Bacteria_p__Verrucomicrobia_c__Verrucomicrobiae_o__Verrucomicrobiales_f__Verrucomicrobiaceae_g__Akkermansia,k__Bacteria_p__Verrucomicrobia_c__[Spartobacteria]_o__[Chthoniobacterales]_f__[Chthoniobacteraceae]_g__DA101,k__Bacteria_p__WPS-2_c___o___f___g__,k__Bacteria_p___c___o___f___g__,comparison_num,comparison_dir
0,douc_VNM_wild_clayton,human_USA,NaN,NaN,NaN,NaN,1.0,1.0,NaN,1.0,...,0.992835,NaN,1.0,NaN,1.0,NaN,NaN,NaN,6,wild-human


In [120]:
douc_0_wild.loc[douc_0_wild.index[0],
                'comparison_num'] = 0

In [121]:
douc_1_wild = combined_pivot_filtered_MCS.loc[
        (combined_pivot_filtered_MCS['comparison_num'] == 6) &
        (combined_pivot_filtered_MCS['comparison_dir'] == 'wild-human')].reset_index(drop=True)

In [122]:
douc_1_wild.loc[douc_0_wild.index[0],
                'comparison_num'] = 1

In [123]:
combined_pivot_filtered_MCS = combined_pivot_filtered_MCS.append([douc_0_wild, douc_1_wild]).reset_index(drop=True)

In [124]:
combined_pivot_filtered_MCS_t = combined_pivot_filtered_MCS.drop(columns=['Group1', 'Group2']).set_index(['comparison_num',
                                                                            'comparison_dir']).transpose().sort_index(axis=1)

In [125]:
combined_pivot_filtered_MCS_t.head()

comparison_num                                                 0             \
comparison_dir                                     captive-human wild-human   
Bacterial_Genus                                                               
Unassigned_____                                              NaN        NaN   
k__Archaea_p__Crenarchaeota_c__Thaumarchaeota_o...           NaN        NaN   
k__Archaea_p__Euryarchaeota_c__Methanobacteria_...           NaN        NaN   
k__Archaea_p__Euryarchaeota_c__Methanobacteria_...           NaN        NaN   
k__Archaea_p__Euryarchaeota_c__Thermoplasmata_o...           1.0        1.0   

comparison_num                                                 1             \
comparison_dir                                     captive-human wild-human   
Bacterial_Genus                                                               
Unassigned_____                                              NaN        NaN   
k__Archaea_p__Crenarchaeota_c__Thaumarchaeota_o...           NaN        NaN   
k__Archaea_p__Euryarchaeota_c__Methanobacteria_...           NaN        NaN   
k__Archaea_p__Euryarchaeota_c__Methanobacteria_...           NaN        NaN   
k__Archaea_p__Euryarchaeota_c__Thermoplasmata_o...           NaN        1.0   

comparison_num                                                 2             \
comparison_dir                                     captive-human wild-human   
Bacterial_Genus                                                               
Unassigned_____                                              NaN        NaN   
k__Archaea_p__Crenarchaeota_c__Thaumarchaeota_o...           NaN        NaN   
k__Archaea_p__Euryarchaeota_c__Methanobacteria_...           NaN        NaN   
k__Archaea_p__Euryarchaeota_c__Methanobacteria_...           NaN        NaN   
k__Archaea_p__Euryarchaeota_c__Thermoplasmata_o...           1.0        1.0   

comparison_num                                                 3             \
comparison_dir                                     captive-human wild-human   
Bacterial_Genus                                                               
Unassigned_____                                              NaN        NaN   
k__Archaea_p__Crenarchaeota_c__Thaumarchaeota_o...           NaN        NaN   
k__Archaea_p__Euryarchaeota_c__Methanobacteria_...           NaN        NaN   
k__Archaea_p__Euryarchaeota_c__Methanobacteria_...           NaN        NaN   
k__Archaea_p__Euryarchaeota_c__Thermoplasmata_o...           1.0        1.0   

comparison_num                                                 4             \
comparison_dir                                     captive-human wild-human   
Bacterial_Genus                                                               
Unassigned_____                                              NaN        NaN   
k__Archaea_p__Crenarchaeota_c__Thaumarchaeota_o...           NaN        NaN   
k__Archaea_p__Euryarchaeota_c__Methanobacteria_...           NaN        NaN   
k__Archaea_p__Euryarchaeota_c__Methanobacteria_...           NaN        NaN   
k__Archaea_p__Euryarchaeota_c__Thermoplasmata_o...           NaN        1.0   

comparison_num                                                 5             \
comparison_dir                                     captive-human wild-human   
Bacterial_Genus                                                               
Unassigned_____                                              NaN        NaN   
k__Archaea_p__Crenarchaeota_c__Thaumarchaeota_o...           NaN        NaN   
k__Archaea_p__Euryarchaeota_c__Methanobacteria_...           NaN        NaN   
k__Archaea_p__Euryarchaeota_c__Methanobacteria_...           NaN        NaN   
k__Archaea_p__Euryarchaeota_c__Thermoplasmata_o...           1.0        1.0   

comparison_num                                                 6             
comparison_dir                                     captive-human wild-human  
Bacterial_Genus                               

In [126]:
target_grps = target_grp_df.copy()

In [127]:
target_grps.drop(columns=['human_population', 'wild_population'], 
                inplace=True)

In [128]:
target_grps

,captive_population
0,douc_USA_captive_clayton
1,douc_SGP_captive_clayton
2,chimp_USA_captive_moeller
3,chimp_USA_captive_campbell
4,howler_CRI_captive_clayton
5,gorilla_USA_captive_campbell
6,douc_VNM_semicaptive_clayton


In [129]:
MCS_df = pd.DataFrame(index=combined_pivot_filtered_MCS_t.index, 
                      columns=target_grps.index)

for i, row in target_grps.iterrows():
    print(row)
    print(i)
    MCS = combined_pivot_filtered_MCS_t.loc[:,(i,
                     'captive-human')]/combined_pivot_filtered_MCS_t.loc[:,(i,
                                          'wild-human')]
    MCS_df[i] = MCS

captive_population    douc_USA_captive_clayton
Name: 0, dtype: object
0
captive_population    douc_SGP_captive_clayton
Name: 1, dtype: object
1
captive_population    chimp_USA_captive_moeller
Name: 2, dtype: object
2
captive_population    chimp_USA_captive_campbell
Name: 3, dtype: object
3
captive_population    howler_CRI_captive_clayton
Name: 4, dtype: object
4
captive_population    gorilla_USA_captive_campbell
Name: 5, dtype: object
5
captive_population    douc_VNM_semicaptive_clayton
Name: 6, dtype: object
6


In [130]:
MCS_df.head()

,0,1,2,3,4,5,6
Bacterial_Genus,,,,,,,
Unassigned_____,NaN,NaN,NaN,NaN,NaN,NaN,NaN
k__Archaea_p__Crenarchaeota_c__Thaumarchaeota_o__Nitrososphaerales_f__Nitrososphaeraceae_g__CandidatusNitrososphaera,NaN,NaN,NaN,NaN,NaN,NaN,NaN
k__Archaea_p__Euryarchaeota_c__Methanobacteria_o__Methanobacteriales_f__Methanobacteriaceae_g__Methanobrevibacter,NaN,NaN,NaN,NaN,NaN,NaN,NaN
k__Archaea_p__Euryarchaeota_c__Methanobacteria_o__Methanobacteriales_f__Methanobacteriaceae_g__Methanosphaera,NaN,NaN,NaN,NaN,NaN,NaN,NaN
k__Archaea_p__Euryarchaeota_c__Thermoplasmata_o__E2_f__[Methanomassiliicoccaceae]_g__vadinCA11,1.0,NaN,1.0,1.0,NaN,1.0,1.0


Make this pretty; add back in the comparison names

In [131]:
MCS_df_t = MCS_df.transpose()
MCS_df_t

Bacterial_Genus,Unassigned_____,k__Archaea_p__Crenarchaeota_c__Thaumarchaeota_o__Nitrososphaerales_f__Nitrososphaeraceae_g__CandidatusNitrososphaera,k__Archaea_p__Euryarchaeota_c__Methanobacteria_o__Methanobacteriales_f__Methanobacteriaceae_g__Methanobrevibacter,k__Archaea_p__Euryarchaeota_c__Methanobacteria_o__Methanobacteriales_f__Methanobacteriaceae_g__Methanosphaera,k__Archaea_p__Euryarchaeota_c__Thermoplasmata_o__E2_f__[Methanomassiliicoccaceae]_g__vadinCA11,k__Bacteria_____,k__Bacteria_p__Acidobacteria_c__Acidobacteria-6_o__iii1-15_f___g__,k__Bacteria_p__Acidobacteria_c__Acidobacteria-6_o__iii1-15_f__mb2424_g__,k__Bacteria_p__Actinobacteria____,k__Bacteria_p__Actinobacteria_c__Acidimicrobiia_o__Acidimicrobiales_f___g__,...,k__Bacteria_p__Tenericutes_c__Mollicutes_o__Mycoplasmatales_f__Mycoplasmataceae_g__,k__Bacteria_p__Tenericutes_c__Mollicutes_o__RF39_f___g__,k__Bacteria_p__Tenericutes_c__RF3_o__ML615J-28_f___g__,k__Bacteria_p__Verrucomicrobia_c__Opitutae_o__HA64_f___g__,k__Bacteria_p__Verrucomicrobia_c__Opitutae_o__[Cerasicoccales]_f__[Cerasicoccaceae]_g__,k__Bacteria_p__Verrucomicrobia_c__Verruco-5_o__WCHB1-41_f__RFP12_g__,k__Bacteria_p__Verrucomicrobia_c__Verrucomicrobiae_o__Verrucomicrobiales_f__Verrucomicrobiaceae_g__Akkermansia,k__Bacteria_p__Verrucomicrobia_c__[Spartobacteria]_o__[Chthoniobacterales]_f__[Chthoniobacteraceae]_g__DA101,k__Bacteria_p__WPS-2_c___o___f___g__,k__Bacteria_p___c___o___f___g__
0,NaN,NaN,NaN,NaN,1.0,0.986176,NaN,NaN,NaN,NaN,...,NaN,1.006445,1.007217,NaN,1.0,NaN,0.364411,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,0.998618,NaN,NaN,NaN,NaN,...,NaN,1.002555,1.007217,NaN,NaN,NaN,0.401949,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,1.0,1.001608,NaN,NaN,NaN,NaN,...,NaN,1.006581,0.985160,NaN,1.0,NaN,0.637753,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,1.0,0.998805,NaN,NaN,NaN,NaN,...,NaN,1.013626,0.980938,NaN,1.0,NaN,NaN,NaN,NaN,1.0
4,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,...,NaN,1.004635,NaN,NaN,NaN,NaN,0.548193,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,1.0,0.994729,NaN,NaN,NaN,NaN,...,NaN,0.998782,0.984683,NaN,1.0,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,1.0,1.000000,NaN,NaN,NaN,NaN,...,NaN,1.002335,1.007217,NaN,1.0,NaN,0.961014,NaN,NaN,NaN


In [132]:
target_grp_df.columns

Index(['captive_population', 'human_population', 'wild_population'], dtype='object')

In [133]:
final_MCS_table = MCS_df_t.merge(target_grp_df,
                      left_index=True,
                      right_index=True).set_index(list(target_grp_df.columns)).transpose()

In [134]:
final_MCS_table.head()

captive_population,douc_USA_captive_clayton,douc_SGP_captive_clayton,chimp_USA_captive_moeller,chimp_USA_captive_campbell,howler_CRI_captive_clayton,gorilla_USA_captive_campbell,douc_VNM_semicaptive_clayton
human_population,human_USA,human_USA,human_USA,human_USA,human_USA,human_USA,human_USA
wild_population,douc_VNM_wild_clayton,douc_VNM_wild_clayton,chimp_TZA_wild_moeller,chimp_DRC_wild_campbell,howler_CRI_wild_clayton,gorilla_DRC_wild_campbell,douc_VNM_wild_clayton
Unassigned_____,NaN,NaN,NaN,NaN,NaN,NaN,NaN
k__Archaea_p__Crenarchaeota_c__Thaumarchaeota_o__Nitrososphaerales_f__Nitrososphaeraceae_g__CandidatusNitrososphaera,NaN,NaN,NaN,NaN,NaN,NaN,NaN
k__Archaea_p__Euryarchaeota_c__Methanobacteria_o__Methanobacteriales_f__Methanobacteriaceae_g__Methanobrevibacter,NaN,NaN,NaN,NaN,NaN,NaN,NaN
k__Archaea_p__Euryarchaeota_c__Methanobacteria_o__Methanobacteriales_f__Methanobacteriaceae_g__Methanosphaera,NaN,NaN,NaN,NaN,NaN,NaN,NaN
k__Archaea_p__Euryarchaeota_c__Thermoplasmata_o__E2_f__[Methanomassiliicoccaceae]_g__vadinCA11,1.0,NaN,1.0,1.0,NaN,1.0,1.0


Write to file

In [135]:
outdir = join('output', 'results')
makedirs(outdir, exist_ok=True)
final_MCS_table.to_csv(join(outdir, 'mcs-amgut.txt'), sep='\t')